## Data preparation

This notebook reads in all the game data (csv files), cleans and transforms it to format fit for analysis. New features are created. 

**Data**
Each csv files contains event level data for one game and one team. Each individual game should thus have two csv files connected to it (one per each team participant to the game)

The data is inputted by hand, creating some errors and inconsistencies.

Data downloaded from: https://www.ultianalytics.com/

### Imports 

In [3]:
import re
import requests as reqs 
import copy
import json
import os
import glob
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import pandas as pd
from pandas.api.types import is_datetime64_any_dtype as is_datetime

import datetime
from datetime import timedelta

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from thefuzz import process

In [4]:
%matplotlib inline

### Links

regex: https://stackoverflow.com/questions/62475160/extract-words-from-string-before-number-python

## Set paths

In [5]:
data = 'data/raw/2019'
data_2018 = 'data/raw/2018'
data_2017 = 'data/raw/2017'
data_2016 = 'data/raw/2016'
data_2015 = 'data/raw/2015'


data_clean = 'data/clean'
parent = '/Users/ilonanietosvaara/Documents/Kurssit/spiced/spiced_projects/scikit-cilantro-student-code/final_project'
datapath = os.path.join(parent, data)
datapath_2018 = os.path.join(parent, data_2018)
datapath_2017 = os.path.join(parent, data_2017)
datapath_2016 = os.path.join(parent, data_2016)
datapath_2015 = os.path.join(parent, data_2015)
datapath_clean = os.path.join(parent, data_clean)

In [6]:
os.chdir(datapath)

## Read in csv files to pandas df


## 2019

In [7]:
def read_csv(files):
    '''
    Read in all csv files to one dataframe, create column team with team name from file name
    
    input: csv files
    output: dataframe
    '''
    df = pd.DataFrame()
    tmp = pd.DataFrame()
    for i, f in enumerate(all_files):
   # for s in re.findall('([a-zA-Z ]*)\d*.*', f):
    #    team_name = s
     #   print(team_name)
        if i == 0:
            try: 
                df = pd.read_csv(f)
                df['team'] = f
                print(f)
            except:
                print(f'===error reading file===: {f}')
                pass
        else:
            try:
                tmp = pd.read_csv(f)
                print(f)
                tmp['team'] = f
                df = df.append(tmp)
            except:
                print(f'===error reading file===: {f}')
                pass
    
    return df


In [8]:
# Place all csv files into list all_files
all_files = glob.glob("*.csv")
all_files

['DetroitMechanix2019-stats.csv',
 'SanDiegoGrowlers2019-stats.csv',
 'DCBreeze2019-stats.csv',
 'AustinSol2019-stats.csv',
 'PhiladelphiaPhoenix2019-stats.csv',
 'TampaBayCannons2019-stats.csv',
 'IndianapolisAlleyCats2019-stats.csv',
 'ChicagoWildfire2019-stats.csv',
 'DallasRoughnecks2019-stats.csv',
 'PittsburghThunderbirds2019-stats.csv',
 'TorontoRush2019-stats.csv',
 'MontrealRoyal2019-stats.csv',
 'NewYorkEmpire2019-stats.csv',
 'AtlantaHustle2019-stats.csv',
 'combined.csv',
 'SeattleCascades2019-stats.csv',
 'MinnesotaWindChill2019-stats.csv',
 'RaleighFlyers2019-stats.csv',
 'MadisonRadicals2019-stats.csv',
 'SanJoseSpiders2019-stats.csv',
 'OttawaOutlaws2019-stats.csv']

In [9]:
df_2019 = read_csv(all_files)

DetroitMechanix2019-stats.csv
SanDiegoGrowlers2019-stats.csv
DCBreeze2019-stats.csv
AustinSol2019-stats.csv
PhiladelphiaPhoenix2019-stats.csv
TampaBayCannons2019-stats.csv
IndianapolisAlleyCats2019-stats.csv
ChicagoWildfire2019-stats.csv
DallasRoughnecks2019-stats.csv
PittsburghThunderbirds2019-stats.csv
TorontoRush2019-stats.csv
MontrealRoyal2019-stats.csv
NewYorkEmpire2019-stats.csv
AtlantaHustle2019-stats.csv


/Users/ilonanietosvaara/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


combined.csv
SeattleCascades2019-stats.csv
MinnesotaWindChill2019-stats.csv
RaleighFlyers2019-stats.csv
MadisonRadicals2019-stats.csv
SanJoseSpiders2019-stats.csv
OttawaOutlaws2019-stats.csv


In [10]:
df_2019.loc[df_2019['Date/Time'].str.len()<10]

NumExpr defaulting to 4 threads.


,Date/Time,Tournamemnt,Opponent,Point Elapsed Seconds,Line,Our Score - End of Point,Their Score - End of Point,Event Type,Action,Passer,...,End Area,End X,End Y,Distance Unit of Measure,Absolute Distance,Lateral Distance,Toward Our Goal Distance,team,Unnamed: 0,Unnamed: 0.1
2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,Pull,NaN,NaN,...,0.610,0.469,yds,94.6,9.1,94.1,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,OpponentCatch,NaN,NaN,...,0.155,0.424,yds,10.7,2.7,-10.3,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,OpponentCatch,NaN,NaN,...,0.308,0.679,yds,18.3,13.6,-12.2,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,Throwaway,NaN,NaN,...,0.518,0.530,yds,18.6,7.9,-16.8,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Offense,Catch,Lemieux,Monfette,...,0.395,0.294,yds,13.5,10.7,8.2,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-29 18:03,AUDL,New York Empire,58,D,19,22,Defense,OpponentCatch,NaN,NaN,...,0.622,0.175,yds,15.3,0.4,-15.3,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
2019-06-29 18:03,AUDL,New York Empire,58,D,19,22,Defense,OpponentCatch,NaN,NaN,...,0.741,0.102,yds,10.3,3.9,-9.6,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
2019-06-29 18:03,AUDL,New York Empire,58,D,19,22,Defense,OpponentCatch,NaN,NaN,...,0.755,0.317,yds,11.5,11.4,-1.1,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
2019-06-29 18:03,AUDL,New York Empire,58,D,19,22,Defense,OpponentCatch,NaN,NaN,...,0.625,0.258,yds,10.9,3.1,10.4,NaN,MontrealRoyal2019-stats.csv,NaN,NaN




for i, f in enumerate(all_files):
   #for s in re.findall('([a-zA-Z ]*)\d*.*', f):
    #    team_name = s
     #   print(team_name)
    if i == 0:
        df = pd.read_csv(f)
        df['team'] = f
        print(f)
    else:
        tmp = pd.read_csv(f)
        tmp['team'] = f
        df = df.append(tmp)

        

## 2018

In [11]:
os.chdir(datapath_2018)
os.getcwd()
# Place all csv files into list all_files
all_files = glob.glob("*.csv")
all_files

['ChicagoWildfire2018-stats.csv',
 'IndianapolisAlleycats2018-stats.csv',
 'DallasRoughnecks2018-stats.csv',
 'PittsburghThunderbirds2018-stats.csv',
 'SanDiegoGrowlers2018-stats.csv',
 'DetroitMechanix2018-stats.csv',
 'DCbreeze2018-stats.csv',
 'AustinSol2018-stats.csv',
 'PhiladelphiaPhoenix2018-stats.csv',
 'MinnesotaWindChill2018-stats.csv',
 'TampaBayCannons-stats.csv',
 'NashvilleNightwatch2018-stats.csv',
 'RaleighFlyers2018-stats.csv',
 'MadisonRadicals2018-stats.csv',
 'SanJoseSpiders2018-stats.csv',
 'OttawaOutlaws2018-stats.csv',
 'TorontoRush2018-stats.csv',
 'MontrealRoyal2018-stats.csv',
 'AtlantaHustle2018-stats.csv',
 'NewYorkEmpire2018-stats.csv',
 'SeattleCascades2018-stats.csv',
 'SanFranciscoFlamethrowers2018-stats.csv']

In [12]:
df_2018 = read_csv(all_files)

ChicagoWildfire2018-stats.csv
IndianapolisAlleycats2018-stats.csv
DallasRoughnecks2018-stats.csv
PittsburghThunderbirds2018-stats.csv
SanDiegoGrowlers2018-stats.csv
DetroitMechanix2018-stats.csv
DCbreeze2018-stats.csv
AustinSol2018-stats.csv
PhiladelphiaPhoenix2018-stats.csv
MinnesotaWindChill2018-stats.csv
TampaBayCannons-stats.csv
NashvilleNightwatch2018-stats.csv
RaleighFlyers2018-stats.csv
MadisonRadicals2018-stats.csv
SanJoseSpiders2018-stats.csv
OttawaOutlaws2018-stats.csv
TorontoRush2018-stats.csv
MontrealRoyal2018-stats.csv
AtlantaHustle2018-stats.csv
NewYorkEmpire2018-stats.csv
SeattleCascades2018-stats.csv
SanFranciscoFlamethrowers2018-stats.csv


In [13]:
df_2018.columns

Index(['Date/Time', 'Tournamemnt', 'Opponent', 'Point Elapsed Seconds', 'Line',
       'Our Score - End of Point', 'Their Score - End of Point', 'Event Type',
       'Action', 'Passer', 'Receiver', 'Defender', 'Hang Time (secs)',
       'Player 0', 'Player 1', 'Player 2', 'Player 3', 'Player 4', 'Player 5',
       'Player 6', 'Player 7', 'Player 8', 'Player 9', 'Player 10',
       'Player 11', 'Player 12', 'Player 13', 'Player 14', 'Player 15',
       'Player 16', 'Player 17', 'Player 18', 'Player 19', 'Player 20',
       'Player 21', 'Player 22', 'Player 23', 'Player 24', 'Player 25',
       'Player 26', 'Player 27', 'Elapsed Time (secs)', 'Begin Area',
       'Begin X', 'Begin Y', 'End Area', 'End X', 'End Y',
       'Distance Unit of Measure', 'Absolute Distance', 'Lateral Distance',
       'Toward Our Goal Distance', 'team'],
      dtype='object')

In [14]:
df_2018['team'].unique()

array(['ChicagoWildfire2018-stats.csv',
       'IndianapolisAlleycats2018-stats.csv',
       'DallasRoughnecks2018-stats.csv',
       'PittsburghThunderbirds2018-stats.csv',
       'SanDiegoGrowlers2018-stats.csv', 'DetroitMechanix2018-stats.csv',
       'DCbreeze2018-stats.csv', 'AustinSol2018-stats.csv',
       'PhiladelphiaPhoenix2018-stats.csv',
       'MinnesotaWindChill2018-stats.csv', 'TampaBayCannons-stats.csv',
       'NashvilleNightwatch2018-stats.csv', 'RaleighFlyers2018-stats.csv',
       'MadisonRadicals2018-stats.csv', 'SanJoseSpiders2018-stats.csv',
       'OttawaOutlaws2018-stats.csv', 'TorontoRush2018-stats.csv',
       'MontrealRoyal2018-stats.csv', 'AtlantaHustle2018-stats.csv',
       'NewYorkEmpire2018-stats.csv', 'SeattleCascades2018-stats.csv',
       'SanFranciscoFlamethrowers2018-stats.csv'], dtype=object)

In [15]:
df_2018.team.isna().sum()

0

### 2017

In [16]:
os.chdir(datapath_2017)

In [17]:
all_files = glob.glob("*.csv")
all_files

['JacksonvilleCannons2017-stats.csv',
 'NashvilleNightwatch2017-stats.csv',
 'MinnesotaWindChill2017-stats.csv',
 'OttawaOutlaws2017-stats.csv',
 'SanJoseSpiders2017-stats.csv',
 'MadisonRadicals2017-stats.csv',
 'RaleighFlyers2017-stats.csv',
 'MontrealRoyal2017-stats.csv',
 'TorontoRush2017-stats.csv',
 'SeattleCascades2017-stats.csv',
 'SanFranciscoFlameThrowers2017-stats.csv',
 'LosAngelesAviators2017-stats.csv',
 'AtlantaHustle2017-stats.csv',
 'NewYorkEmpire2017-stats.csv',
 'ChicagoWildfire2017-stats.csv',
 'IndianapolisAlleyCats2017-stats.csv',
 'VancouverRiptide2017-stats.csv',
 'PittsburghThunderbirds2017-stats.csv',
 'DallasRoughnecks2017-stats.csv',
 'DCBreeze2017-stats.csv',
 'SanDiegoGrowlers2017-stats.csv',
 'DetroitMechanix2017-stats.csv',
 'PhiladelphiaPhoenix2017-stats.csv',
 'AustinSol2017-stats.csv']

In [18]:
df_2017 = read_csv(all_files)

JacksonvilleCannons2017-stats.csv
NashvilleNightwatch2017-stats.csv
MinnesotaWindChill2017-stats.csv
OttawaOutlaws2017-stats.csv
SanJoseSpiders2017-stats.csv
===error reading file===: MadisonRadicals2017-stats.csv
RaleighFlyers2017-stats.csv
MontrealRoyal2017-stats.csv
TorontoRush2017-stats.csv
SeattleCascades2017-stats.csv
SanFranciscoFlameThrowers2017-stats.csv
===error reading file===: LosAngelesAviators2017-stats.csv
AtlantaHustle2017-stats.csv
NewYorkEmpire2017-stats.csv
ChicagoWildfire2017-stats.csv
IndianapolisAlleyCats2017-stats.csv
VancouverRiptide2017-stats.csv
PittsburghThunderbirds2017-stats.csv
DallasRoughnecks2017-stats.csv
DCBreeze2017-stats.csv
SanDiegoGrowlers2017-stats.csv
DetroitMechanix2017-stats.csv
PhiladelphiaPhoenix2017-stats.csv
AustinSol2017-stats.csv


### 2016

In [19]:
os.chdir(datapath_2016)
os.getcwd()
# Place all csv files into list all_files
all_files = glob.glob("*.csv")
all_files

['MontrealRoyal2016-stats.csv',
 'TorontoRush2016-stats.csv',
 'SeattleCascades2016-stats.csv',
 'CincinnatiRevolution-stats.csv',
 'NewYorkEmpire2016-stats.csv',
 'LosAngelesAviators2016-stats.csv',
 'AtlantaHustle2016-stats.csv',
 'SFFlameThrowers-stats.csv',
 'NashvilleNightwatch2016-stats.csv',
 'JacksonvilleCannons2016-stats.csv',
 'MinnesotaWindChill2016-stats.csv',
 'OttawaOutlaws2016-stats.csv',
 'SanJoseSpiders2016-stats.csv',
 'MadisonRadicals2016-stats.csv',
 'RaleighFlyers2016-stats.csv',
 'DCBreeze2016-stats.csv',
 'DetroitMechanix2016-stats.csv',
 'CharlotteExpress2016-stats.csv',
 'SanDiegoGrowlers2016-stats.csv',
 'PhiladelphiaPhoenix2016-stats.csv',
 'AustinSol2016-stats.csv',
 'IndianapolisAlleyCats2016-stats.csv',
 'ChicagoWildfire2016-stats.csv',
 'PittsburghThunderbirds2016-stats.csv',
 'VancouverRiptide2016-stats.csv',
 'DallasRoughnecks2016-stats.csv']

In [20]:
df_2016 = read_csv(all_files)

===error reading file===: MontrealRoyal2016-stats.csv
TorontoRush2016-stats.csv
SeattleCascades2016-stats.csv
CincinnatiRevolution-stats.csv
NewYorkEmpire2016-stats.csv
LosAngelesAviators2016-stats.csv
AtlantaHustle2016-stats.csv
SFFlameThrowers-stats.csv
NashvilleNightwatch2016-stats.csv
JacksonvilleCannons2016-stats.csv
MinnesotaWindChill2016-stats.csv
OttawaOutlaws2016-stats.csv
SanJoseSpiders2016-stats.csv
MadisonRadicals2016-stats.csv
RaleighFlyers2016-stats.csv
DCBreeze2016-stats.csv
DetroitMechanix2016-stats.csv
CharlotteExpress2016-stats.csv
SanDiegoGrowlers2016-stats.csv
PhiladelphiaPhoenix2016-stats.csv
AustinSol2016-stats.csv
IndianapolisAlleyCats2016-stats.csv
ChicagoWildfire2016-stats.csv
PittsburghThunderbirds2016-stats.csv
VancouverRiptide2016-stats.csv
DallasRoughnecks2016-stats.csv


### 2015

In [21]:
os.chdir(datapath_2015)
os.getcwd()
# Place all csv files into list all_files
all_files = glob.glob("*.csv")
all_files

['ChicagoWildfire2015-stats.csv',
 'IndianapolisAlleyCats2015-stats.csv',
 'LosAngelesAviators2016-stats.csv',
 'VancouverRiptide2015-stats.csv',
 'PittsburghThunderbirds2015-stats.csv',
 'SanDiegoGrowlers2015-stats.csv',
 'DetroitMechanix2015-stats.csv',
 'CharlotteExpress2015-stats.csv',
 'DCBreeze2015-stats.csv',
 'PhiladelphiaPhoenix2015-stats.csv',
 'JacksonvilleCannons2015-stats.csv',
 'NashvilleNightwatch2015-stats.csv',
 'MinnesotaWindChill2015-stats.csv',
 'SanJoseSpiders2015-stats.csv',
 'OttawaOutlaws2015-stats.csv',
 'RaleighFlyers2015-stats.csv',
 'MadisonRadicals2015-stats.csv',
 'MontrealRoyal2015-stats.csv',
 'TorontoRush2015-stats.csv',
 'SeattleCascades2015-stats.csv',
 'SanFranciscoFlameThrowers2015-stats.csv',
 'RochesterDragons2015-stats.csv',
 'CincinnatiRevolution2015-stats.csv',
 'NewYorkEmpire2015-stats.csv']

In [22]:
df_2015 = read_csv(all_files)

ChicagoWildfire2015-stats.csv
IndianapolisAlleyCats2015-stats.csv
LosAngelesAviators2016-stats.csv
VancouverRiptide2015-stats.csv
PittsburghThunderbirds2015-stats.csv
SanDiegoGrowlers2015-stats.csv
DetroitMechanix2015-stats.csv
CharlotteExpress2015-stats.csv
DCBreeze2015-stats.csv
PhiladelphiaPhoenix2015-stats.csv
JacksonvilleCannons2015-stats.csv
NashvilleNightwatch2015-stats.csv
MinnesotaWindChill2015-stats.csv
SanJoseSpiders2015-stats.csv
OttawaOutlaws2015-stats.csv
RaleighFlyers2015-stats.csv
MadisonRadicals2015-stats.csv
MontrealRoyal2015-stats.csv
TorontoRush2015-stats.csv
SeattleCascades2015-stats.csv
SanFranciscoFlameThrowers2015-stats.csv
RochesterDragons2015-stats.csv
CincinnatiRevolution2015-stats.csv
NewYorkEmpire2015-stats.csv


### Append all years into one

In [23]:
df = pd.concat([df_2019, df_2018, df_2017, df_2016, df_2015], ignore_index=False)

In [24]:
#df.loc[df['Date/Time'].str.len()<10]

In [25]:
df.shape

(738279, 55)

In [26]:
# Check memory use 
df.info(verbose = False)

<class 'pandas.core.frame.DataFrame'>
Index: 738279 entries, 0 to 5386
Columns: 55 entries, Date/Time to Unnamed: 0.1
dtypes: float64(18), int64(1), object(36)
memory usage: 315.4+ MB


### Check for missing values

In [27]:
df.isna().sum()

Date/Time                       7459
Tournamemnt                   193850
Opponent                           0
Point Elapsed Seconds              0
Line                               0
Our Score - End of Point           0
Their Score - End of Point         0
Event Type                         0
Action                         13615
Passer                        163058
Receiver                      171072
Defender                      596606
Hang Time (secs)              662908
Player 0                         937
Player 1                        1059
Player 2                        1146
Player 3                        1227
Player 4                        1341
Player 5                        1745
Player 6                       35686
Player 7                      661084
Player 8                      679855
Player 9                      683720
Player 10                     686892
Player 11                     690555
Player 12                     695890
Player 13                     707247
P

### clean columns names

In [28]:
def clean_cols(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_', regex=True).str.replace('(', '', regex=True).str.replace(')', '', regex=True)
    df.columns = df.columns.str.lower().str.replace('/', '', regex=True).str.replace('_-_', '_', regex=True)
    return df

In [29]:
df = clean_cols(df)

In [30]:
df.columns

Index(['datetime', 'tournamemnt', 'opponent', 'point_elapsed_seconds', 'line',
       'our_score_end_of_point', 'their_score_end_of_point', 'event_type',
       'action', 'passer', 'receiver', 'defender', 'hang_time_secs',
       'player_0', 'player_1', 'player_2', 'player_3', 'player_4', 'player_5',
       'player_6', 'player_7', 'player_8', 'player_9', 'player_10',
       'player_11', 'player_12', 'player_13', 'player_14', 'player_15',
       'player_16', 'player_17', 'player_18', 'player_19', 'player_20',
       'player_21', 'player_22', 'player_23', 'player_24', 'player_25',
       'player_26', 'player_27', 'elapsed_time_secs', 'begin_area', 'begin_x',
       'begin_y', 'end_area', 'end_x', 'end_y', 'distance_unit_of_measure',
       'absolute_distance', 'lateral_distance', 'toward_our_goal_distance',
       'team', 'unnamed:_0', 'unnamed:_0.1'],
      dtype='object')

In [31]:
df['game_id_str'] = df['datetime'] + '_' + df['team']

In [30]:
df['game_id_str2'] = df['game_id_str']

In [31]:
# Check memory use 
df.info(verbose = False)

<class 'pandas.core.frame.DataFrame'>
Index: 738279 entries, 0 to 5386
Columns: 57 entries, datetime to game_id_str2
dtypes: float64(18), int64(1), object(38)
memory usage: 326.7+ MB


In [32]:
df

,datetime,tournamemnt,opponent,point_elapsed_seconds,line,our_score_end_of_point,their_score_end_of_point,event_type,action,passer,...,end_y,distance_unit_of_measure,absolute_distance,lateral_distance,toward_our_goal_distance,team,unnamed:_0,unnamed:_0.1,game_id_str,game_id_str2
0,2019-04-20 18:52,AUDL,Indianapolis AlleyCats,6,D,0,1,Defense,Pull,NaN,...,NaN,NaN,NaN,NaN,NaN,DetroitMechanix2019-stats.csv,NaN,NaN,2019-04-20 18:52_DetroitMechanix2019-stats.csv,2019-04-20 18:52_DetroitMechanix2019-stats.csv
1,2019-04-20 18:52,AUDL,Indianapolis AlleyCats,6,D,0,1,Defense,Goal,NaN,...,NaN,NaN,NaN,NaN,NaN,DetroitMechanix2019-stats.csv,NaN,NaN,2019-04-20 18:52_DetroitMechanix2019-stats.csv,2019-04-20 18:52_DetroitMechanix2019-stats.csv
2,2019-04-20 18:52,AUDL,Indianapolis AlleyCats,59,O,0,2,Offense,Catch,Cubitt,...,NaN,NaN,NaN,NaN,NaN,DetroitMechanix2019-stats.csv,NaN,NaN,2019-04-20 18:52_DetroitMechanix2019-stats.csv,2019-04-20 18:52_DetroitMechanix2019-stats.csv
3,2019-04-20 18:52,AUDL,Indianapolis AlleyCats,59,O,0,2,Offense,Catch,Innis,...,NaN,NaN,NaN,NaN,NaN,DetroitMechanix2019-stats.csv,NaN,NaN,2019-04-20 18:52_DetroitMechanix2019-stats.csv,2019-04-20 18:52_DetroitMechanix2019-stats.csv
4,2019-04-20 18:52,AUDL,Indianapolis AlleyCats,59,O,0,2,Offense,Catch,Bert,...,NaN,NaN,NaN,NaN,NaN,DetroitMechanix2019-stats.csv,NaN,NaN,2019-04-20 18:52_DetroitMechanix2019-stats.csv,2019-04-20 18:52_DetroitMechanix2019-stats.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5382,2015-05-09 19:03,AUDL,Totonto Rush,27,O,25,28,Offense,Catch,Skware K,...,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv,2015-05-09 19:03_NewYorkEmpire2015-stats.csv
5383,2015-05-09 19:03,AUDL,Totonto Rush,27,O,25,28,Offense,Catch,Aulett M,...,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv,2015-05-09 19:03_NewYorkEmpire2015-stats.csv
5384,2015-05-09 19:03,AUDL,Totonto Rush,27,O,25,28,Offense,Goal,Marsh J,...,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv,2015-05-09 19:03_NewYorkEmpire2015-stats.csv
5385,2015-05-09 19:03,AUDL,Totonto Rush,19,D,25,28,Defense,Pull,NaN,...,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv,2015-05-09 19:03_NewYorkEmpire2015-stats.csv


In [33]:
# Check shifted values 
df.line.value_counts()

O     441826
D     261464
6       1763
7       1706
4       1702
9       1648
5       1609
8       1584
16      1521
13      1509
2       1508
14      1506
11      1488
15      1482
12      1473
1       1445
10      1442
3       1392
17      1234
19      1214
18      1111
20      1048
21      1011
23       989
22       777
24       644
0        374
25       367
26       345
27       233
29       185
28       173
30       136
32        97
31        92
34        49
35        48
33        40
39        17
38        14
36         8
37         5
Name: line, dtype: int64

In [34]:
df.datetime.value_counts()

AUDL                25832
2019-05-18 19:10     2253
2019-05-11 19:03     2157
2019-05-04 18:38     2133
2019-06-22 18:13     2103
                    ...  
2017-06-24 18:35      116
2016-05-06 19:03      112
2018-07-08 14:00       94
2018-07-08 13:06       87
2017-04-01 18:04       38
Name: datetime, Length: 1420, dtype: int64

In [35]:
# Check how many columns shifter 
#df.loc[(df.line!= 'O') | (df.line!='D')].sum()

### Drop columns with more than 70% missing values

In [36]:
#df = df.dropna(thresh=df.shape[0]*0.3,how='all', axis=1)

In [37]:
len(df.columns)

57

In [38]:
df.columns

Index(['datetime', 'tournamemnt', 'opponent', 'point_elapsed_seconds', 'line',
       'our_score_end_of_point', 'their_score_end_of_point', 'event_type',
       'action', 'passer', 'receiver', 'defender', 'hang_time_secs',
       'player_0', 'player_1', 'player_2', 'player_3', 'player_4', 'player_5',
       'player_6', 'player_7', 'player_8', 'player_9', 'player_10',
       'player_11', 'player_12', 'player_13', 'player_14', 'player_15',
       'player_16', 'player_17', 'player_18', 'player_19', 'player_20',
       'player_21', 'player_22', 'player_23', 'player_24', 'player_25',
       'player_26', 'player_27', 'elapsed_time_secs', 'begin_area', 'begin_x',
       'begin_y', 'end_area', 'end_x', 'end_y', 'distance_unit_of_measure',
       'absolute_distance', 'lateral_distance', 'toward_our_goal_distance',
       'team', 'unnamed:_0', 'unnamed:_0.1', 'game_id_str', 'game_id_str2'],
      dtype='object')

In [39]:
df.tournamemnt.unique()

array(['AUDL', nan, 'AUDL 2019', 'Home Game', 'Ottawa Outlaws',
       'Toronto Rush', 'DC Breeze', 'Philadelphia Phoenix',
       'New York Empire', 'AUDL West', 'AUDL 2018', 'AUDL 2018 Playoffs',
       'Playoff Week 1', 'Audl Game 3 Against Montreal', 'Audl Game 2',
       '2018 Season', 'AUDL Playoffs', '6/3', 'Audl', 'AUDL West 2018',
       'NYC Empire', 'Toronto RUSH', 'AUDL West 2017', 'AUDL 2017 Final',
       'AUDL 2017 Semifinal', '5-20', '5-21', '6-25', '6/24', 'AUDL 2017',
       'audl', 'Seattle Cascades', 'AUDL West 2016', 'Audl Playoffs',
       'AUDL PLayoffs', 'AUDL 2016 Playoffs', 'AUDL 2016', 'AUDL 2015',
       'San Francisco Flamethrowers', 'Madison Radicals',
       'Vancouver Riptide', 'San Diego Growlers', 'Los Angeles Aviators',
       'San Francisco FlameThrowers', 'Regular Season Game 1',
       'Pittsburgh Thunderbirds', 'Indianapolis AlleyCays',
       'Minnesota Windchill', 'Detroit Mechanix', 'Cincinatti Revolution',
       'Raleigh Flyers', 'Chicago Wil

non_tournaments = ['San Francisco Flamethrowers', 'Madison Radicals',
       'Vancouver Riptide', 'San Diego Growlers', 'Los Angeles Aviators',
       'San Francisco FlameThrowers', 'Regular Season Game 1',
       'Pittsburgh Thunderbirds', 'Indianapolis AlleyCays',
       'Minnesota Windchill', 'Detroit Mechanix', 'Cincinatti Revolution',
       'Raleigh Flyers', 'Chicago Wildfire', 'San Jose Spiders',
       'Indianapolis alleycats', 'Chicag Wildfire', 'Ottawa Outlaws',
       'Toronto Rush', 'DC Breeze', 'Philadelphia Phoenix',
       'New York Empire','NYC Empire', 'Toronto RUSH',]

In [40]:
df_notpure = df.loc[(df.line !='O') & (df.line != 'D')].copy()

In [41]:
#df_notpure = df.loc[(df.tournamemnt !='AUDL') & (df.tournamemnt.isnull() == False) & (df.tournamemnt !='AUDL 2019') & (df.tournamemnt !='AUDL West') & (df.tournamemnt !='Home Game')].copy()

In [42]:
df_notpure.tournamemnt.unique()

array(['Ottawa Outlaws', 'Toronto Rush', 'DC Breeze',
       'Philadelphia Phoenix', 'New York Empire', 'NYC Empire',
       'Toronto RUSH', 'San Francisco Flamethrowers', 'Madison Radicals',
       'Vancouver Riptide', 'San Diego Growlers', 'Los Angeles Aviators',
       'San Francisco FlameThrowers', 'Seattle Cascades',
       'Pittsburgh Thunderbirds', 'Indianapolis AlleyCays',
       'Minnesota Windchill', 'Detroit Mechanix', 'Cincinatti Revolution',
       'Raleigh Flyers', 'Chicago Wildfire', 'San Jose Spiders',
       'Indianapolis alleycats', 'Chicag Wildfire'], dtype=object)

In [43]:
df_pure = df.loc[(df.line =='O') | (df.line == 'D')].copy()

In [44]:
#df_pure = df.loc[(df.tournamemnt =='AUDL') | (df.tournamemnt.isnull() == True) | (df.tournamemnt =='AUDL 2019') | (df.tournamemnt =='AUDL West') | (df.tournamemnt =='Home Game')].copy()

In [45]:
df_pure.tournamemnt.unique()

array(['AUDL', nan, 'AUDL 2019', 'Home Game', 'AUDL West', 'AUDL 2018',
       'AUDL 2018 Playoffs', 'Playoff Week 1',
       'Audl Game 3 Against Montreal', 'Audl Game 2', '2018 Season',
       'AUDL Playoffs', '6/3', 'Audl', 'AUDL West 2018', 'AUDL West 2017',
       'AUDL 2017 Final', 'AUDL 2017 Semifinal', '5-20', '5-21', '6-25',
       '6/24', 'AUDL 2017', 'audl', 'Seattle Cascades', 'AUDL West 2016',
       'Audl Playoffs', 'AUDL PLayoffs', 'AUDL 2016 Playoffs',
       'AUDL 2016', 'AUDL 2015', 'Regular Season Game 1', '20 Juin 2015',
       '21 Juin 2015', 'AUDL West 2015'], dtype=object)

In [46]:
df_pure.loc[df_pure['tournamemnt'].isna()]

,datetime,tournamemnt,opponent,point_elapsed_seconds,line,our_score_end_of_point,their_score_end_of_point,event_type,action,passer,...,end_y,distance_unit_of_measure,absolute_distance,lateral_distance,toward_our_goal_distance,team,unnamed:_0,unnamed:_0.1,game_id_str,game_id_str2
1422,2019-04-06 18:51,NaN,Los Angeles Aviators,52,O,1,0,Offense,Catch,Gutkowsk,...,NaN,NaN,NaN,NaN,NaN,SanDiegoGrowlers2019-stats.csv,NaN,NaN,2019-04-06 18:51_SanDiegoGrowlers2019-stats.csv,2019-04-06 18:51_SanDiegoGrowlers2019-stats.csv
1423,2019-04-06 18:51,NaN,Los Angeles Aviators,52,O,1,0,Offense,Catch,Tran,...,NaN,NaN,NaN,NaN,NaN,SanDiegoGrowlers2019-stats.csv,NaN,NaN,2019-04-06 18:51_SanDiegoGrowlers2019-stats.csv,2019-04-06 18:51_SanDiegoGrowlers2019-stats.csv
1424,2019-04-06 18:51,NaN,Los Angeles Aviators,52,O,1,0,Offense,Catch,Milardov,...,NaN,NaN,NaN,NaN,NaN,SanDiegoGrowlers2019-stats.csv,NaN,NaN,2019-04-06 18:51_SanDiegoGrowlers2019-stats.csv,2019-04-06 18:51_SanDiegoGrowlers2019-stats.csv
1425,2019-04-06 18:51,NaN,Los Angeles Aviators,52,O,1,0,Offense,Catch,Tran,...,NaN,NaN,NaN,NaN,NaN,SanDiegoGrowlers2019-stats.csv,NaN,NaN,2019-04-06 18:51_SanDiegoGrowlers2019-stats.csv,2019-04-06 18:51_SanDiegoGrowlers2019-stats.csv
1426,2019-04-06 18:51,NaN,Los Angeles Aviators,52,O,1,0,Offense,Catch,Leggio,...,NaN,NaN,NaN,NaN,NaN,SanDiegoGrowlers2019-stats.csv,NaN,NaN,2019-04-06 18:51_SanDiegoGrowlers2019-stats.csv,2019-04-06 18:51_SanDiegoGrowlers2019-stats.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4477,2015-07-16 17:55,NaN,Toronto Rush,631,O,17,38,Offense,Throwaway,Max R,...,NaN,NaN,NaN,NaN,NaN,RochesterDragons2015-stats.csv,NaN,NaN,2015-07-16 17:55_RochesterDragons2015-stats.csv,2015-07-16 17:55_RochesterDragons2015-stats.csv
4478,2015-07-16 17:55,NaN,Toronto Rush,631,O,17,38,Cessation,EndOfFirstQuarter,NaN,...,NaN,NaN,NaN,NaN,NaN,RochesterDragons2015-stats.csv,NaN,NaN,2015-07-16 17:55_RochesterDragons2015-stats.csv,2015-07-16 17:55_RochesterDragons2015-stats.csv
4479,2015-07-16 17:55,NaN,Toronto Rush,7,D,17,38,Cessation,Halftime,NaN,...,NaN,NaN,NaN,NaN,NaN,RochesterDragons2015-stats.csv,NaN,NaN,2015-07-16 17:55_RochesterDragons2015-stats.csv,2015-07-16 17:55_RochesterDragons2015-stats.csv
4480,2015-07-16 17:55,NaN,Toronto Rush,7,O,17,38,Cessation,EndOfThirdQuarter,NaN,...,NaN,NaN,NaN,NaN,NaN,RochesterDragons2015-stats.csv,NaN,NaN,2015-07-16 17:55_RochesterDragons2015-stats.csv,2015-07-16 17:55_RochesterDragons2015-stats.csv


### Shift cells to right one on df_notpure

In [47]:
df_notpure = df_notpure.shift(periods=1, axis=1)

In [48]:
df_notpure['datetime'] = df_notpure.index

In [49]:
df_notpure.reset_index(drop=True, inplace=True)

In [50]:
df_notpure

,datetime,tournamemnt,opponent,point_elapsed_seconds,line,our_score_end_of_point,their_score_end_of_point,event_type,action,passer,...,end_y,distance_unit_of_measure,absolute_distance,lateral_distance,toward_our_goal_distance,team,unnamed:_0,unnamed:_0.1,game_id_str,game_id_str2
0,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,Pull,NaN,...,0.469,yds,94.6,9.1,94.1,NaN,MontrealRoyal2019-stats.csv,NaN,NaN,AUDL_MontrealRoyal2019-stats.csv
1,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,OpponentCatch,NaN,...,0.424,yds,10.7,2.7,-10.3,NaN,MontrealRoyal2019-stats.csv,NaN,NaN,AUDL_MontrealRoyal2019-stats.csv
2,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,OpponentCatch,NaN,...,0.679,yds,18.3,13.6,-12.2,NaN,MontrealRoyal2019-stats.csv,NaN,NaN,AUDL_MontrealRoyal2019-stats.csv
3,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,Throwaway,NaN,...,0.530,yds,18.6,7.9,-16.8,NaN,MontrealRoyal2019-stats.csv,NaN,NaN,AUDL_MontrealRoyal2019-stats.csv
4,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Offense,Catch,Lemieux,...,0.294,yds,13.5,10.7,8.2,NaN,MontrealRoyal2019-stats.csv,NaN,NaN,AUDL_MontrealRoyal2019-stats.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34984,2015-05-22 19:01,NaN,Minnesota Windchill,34,D,27,10,Defense,OpponentCatch,NaN,...,0.354,yds,8.5,8.3,-1.5,NaN,MadisonRadicals2015-stats.csv,NaN,NaN,NaN
34985,2015-05-22 19:01,NaN,Minnesota Windchill,34,D,27,10,Defense,OpponentCatch,NaN,...,0.576,yds,14.0,11.9,7.4,NaN,MadisonRadicals2015-stats.csv,NaN,NaN,NaN
34986,2015-05-22 19:01,NaN,Minnesota Windchill,34,D,27,10,Defense,OpponentCatch,NaN,...,0.733,yds,17.2,8.4,-15.1,NaN,MadisonRadicals2015-stats.csv,NaN,NaN,NaN
34987,2015-05-22 19:01,NaN,Minnesota Windchill,34,D,27,10,Defense,OpponentCatch,NaN,...,0.781,yds,13.8,2.5,-13.6,NaN,MadisonRadicals2015-stats.csv,NaN,NaN,NaN


In [51]:
df_notpure['team'] = df_notpure['game_id_str']

In [52]:
df_notpure['game_id_str'] = df_notpure['datetime'].astype('str') + '_' + df_notpure['team']


In [53]:
df_notpure.columns

Index(['datetime', 'tournamemnt', 'opponent', 'point_elapsed_seconds', 'line',
       'our_score_end_of_point', 'their_score_end_of_point', 'event_type',
       'action', 'passer', 'receiver', 'defender', 'hang_time_secs',
       'player_0', 'player_1', 'player_2', 'player_3', 'player_4', 'player_5',
       'player_6', 'player_7', 'player_8', 'player_9', 'player_10',
       'player_11', 'player_12', 'player_13', 'player_14', 'player_15',
       'player_16', 'player_17', 'player_18', 'player_19', 'player_20',
       'player_21', 'player_22', 'player_23', 'player_24', 'player_25',
       'player_26', 'player_27', 'elapsed_time_secs', 'begin_area', 'begin_x',
       'begin_y', 'end_area', 'end_x', 'end_y', 'distance_unit_of_measure',
       'absolute_distance', 'lateral_distance', 'toward_our_goal_distance',
       'team', 'unnamed:_0', 'unnamed:_0.1', 'game_id_str', 'game_id_str2'],
      dtype='object')

In [54]:
df_notpure.team.unique()

array([nan])

In [55]:
df_pure.reset_index(drop=True, inplace=True)

In [56]:
df_pure

,datetime,tournamemnt,opponent,point_elapsed_seconds,line,our_score_end_of_point,their_score_end_of_point,event_type,action,passer,...,end_y,distance_unit_of_measure,absolute_distance,lateral_distance,toward_our_goal_distance,team,unnamed:_0,unnamed:_0.1,game_id_str,game_id_str2
0,2019-04-20 18:52,AUDL,Indianapolis AlleyCats,6,D,0,1,Defense,Pull,NaN,...,NaN,NaN,NaN,NaN,NaN,DetroitMechanix2019-stats.csv,NaN,NaN,2019-04-20 18:52_DetroitMechanix2019-stats.csv,2019-04-20 18:52_DetroitMechanix2019-stats.csv
1,2019-04-20 18:52,AUDL,Indianapolis AlleyCats,6,D,0,1,Defense,Goal,NaN,...,NaN,NaN,NaN,NaN,NaN,DetroitMechanix2019-stats.csv,NaN,NaN,2019-04-20 18:52_DetroitMechanix2019-stats.csv,2019-04-20 18:52_DetroitMechanix2019-stats.csv
2,2019-04-20 18:52,AUDL,Indianapolis AlleyCats,59,O,0,2,Offense,Catch,Cubitt,...,NaN,NaN,NaN,NaN,NaN,DetroitMechanix2019-stats.csv,NaN,NaN,2019-04-20 18:52_DetroitMechanix2019-stats.csv,2019-04-20 18:52_DetroitMechanix2019-stats.csv
3,2019-04-20 18:52,AUDL,Indianapolis AlleyCats,59,O,0,2,Offense,Catch,Innis,...,NaN,NaN,NaN,NaN,NaN,DetroitMechanix2019-stats.csv,NaN,NaN,2019-04-20 18:52_DetroitMechanix2019-stats.csv,2019-04-20 18:52_DetroitMechanix2019-stats.csv
4,2019-04-20 18:52,AUDL,Indianapolis AlleyCats,59,O,0,2,Offense,Catch,Bert,...,NaN,NaN,NaN,NaN,NaN,DetroitMechanix2019-stats.csv,NaN,NaN,2019-04-20 18:52_DetroitMechanix2019-stats.csv,2019-04-20 18:52_DetroitMechanix2019-stats.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703285,2015-05-09 19:03,AUDL,Totonto Rush,27,O,25,28,Offense,Catch,Skware K,...,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv,2015-05-09 19:03_NewYorkEmpire2015-stats.csv
703286,2015-05-09 19:03,AUDL,Totonto Rush,27,O,25,28,Offense,Catch,Aulett M,...,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv,2015-05-09 19:03_NewYorkEmpire2015-stats.csv
703287,2015-05-09 19:03,AUDL,Totonto Rush,27,O,25,28,Offense,Goal,Marsh J,...,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv,2015-05-09 19:03_NewYorkEmpire2015-stats.csv
703288,2015-05-09 19:03,AUDL,Totonto Rush,19,D,25,28,Defense,Pull,NaN,...,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv,2015-05-09 19:03_NewYorkEmpire2015-stats.csv


In [57]:
df_pure.game_id_str.nunique()

1777

In [58]:
df_notpure.game_id_str.nunique()

0

In [59]:
df = pd.concat([df_notpure, df_pure], axis=0)

In [60]:
df.drop('game_id_str2', inplace=True, axis=1)

In [61]:
df

,datetime,tournamemnt,opponent,point_elapsed_seconds,line,our_score_end_of_point,their_score_end_of_point,event_type,action,passer,...,end_x,end_y,distance_unit_of_measure,absolute_distance,lateral_distance,toward_our_goal_distance,team,unnamed:_0,unnamed:_0.1,game_id_str
0,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,Pull,NaN,...,0.610,0.469,yds,94.6,9.1,94.1,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
1,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,OpponentCatch,NaN,...,0.155,0.424,yds,10.7,2.7,-10.3,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
2,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,OpponentCatch,NaN,...,0.308,0.679,yds,18.3,13.6,-12.2,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
3,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,Throwaway,NaN,...,0.518,0.53,yds,18.6,7.9,-16.8,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
4,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Offense,Catch,Lemieux,...,0.395,0.294,yds,13.5,10.7,8.2,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703285,2015-05-09 19:03,AUDL,Totonto Rush,27,O,25,28,Offense,Catch,Skware K,...,NaN,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv
703286,2015-05-09 19:03,AUDL,Totonto Rush,27,O,25,28,Offense,Catch,Aulett M,...,NaN,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv
703287,2015-05-09 19:03,AUDL,Totonto Rush,27,O,25,28,Offense,Goal,Marsh J,...,NaN,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv
703288,2015-05-09 19:03,AUDL,Totonto Rush,19,D,25,28,Defense,Pull,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv


In [62]:
df.line.unique()

array(['D', 'O'], dtype=object)

In [63]:
df.hang_time_secs.unique()

array([nan, 7.848, 7.134, ..., 8.502, 9.887, 8.061], dtype=object)

In [64]:
df.elapsed_time_secs.unique()

array([0.0, 4.0, 8.0, ..., 10014, 10216, 3300273], dtype=object)

### Drop unnecessary columns


df.drop(['begin_x', 'begin_area',
       'distance_unit_of_measure','end_x','lateral_distance',
       'end_y', 'absolute_distance', 'toward_our_goal_distance', 'begin_y','end_area'], axis=1, inplace=True)

#Drop player columns
df.drop(['player_0', 'player_1',
       'player_2', 'player_3', 'player_4', 'player_5', 'player_6', 'player_7',
       'player_8', 'player_9', 'player_10', 'player_11', 'player_12',
       'player_13', 'player_14', 'player_15', 'player_16', 'player_17',
       'player_18', 'player_19', 'player_20', 'player_21', 'player_22',
       'player_23', 'player_24', 'player_25', 'player_26', 'player_27'], axis=1, inplace=True)

In [65]:
# Check datetime columns is ok
min(df.datetime.str.len())
max(df.datetime.str.len())

16

In [71]:
df.reset_index(drop=True, inplace=True)

In [72]:
df

,datetime,tournamemnt,opponent,point_elapsed_seconds,line,our_score_end_of_point,their_score_end_of_point,event_type,action,passer,...,end_x,end_y,distance_unit_of_measure,absolute_distance,lateral_distance,toward_our_goal_distance,team,unnamed:_0,unnamed:_0.1,game_id_str
0,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,Pull,NaN,...,0.610,0.469,yds,94.6,9.1,94.1,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
1,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,OpponentCatch,NaN,...,0.155,0.424,yds,10.7,2.7,-10.3,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
2,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,OpponentCatch,NaN,...,0.308,0.679,yds,18.3,13.6,-12.2,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
3,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Defense,Throwaway,NaN,...,0.518,0.53,yds,18.6,7.9,-16.8,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
4,2019-07-04 19:12,AUDL,Ottawa Outlaws,30,D,1,0,Offense,Catch,Lemieux,...,0.395,0.294,yds,13.5,10.7,8.2,NaN,MontrealRoyal2019-stats.csv,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738274,2015-05-09 19:03,AUDL,Totonto Rush,27,O,25,28,Offense,Catch,Skware K,...,NaN,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv
738275,2015-05-09 19:03,AUDL,Totonto Rush,27,O,25,28,Offense,Catch,Aulett M,...,NaN,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv
738276,2015-05-09 19:03,AUDL,Totonto Rush,27,O,25,28,Offense,Goal,Marsh J,...,NaN,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv
738277,2015-05-09 19:03,AUDL,Totonto Rush,19,D,25,28,Defense,Pull,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NewYorkEmpire2015-stats.csv,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv


In [68]:
df.shape

(738279, 56)

In [69]:
df.event_type.unique()

array(['Defense', 'Offense', 'Cessation', 'Cessastion'], dtype=object)

In [70]:
df.dtypes

datetime                     object
tournamemnt                  object
opponent                     object
point_elapsed_seconds        object
line                         object
our_score_end_of_point       object
their_score_end_of_point     object
event_type                   object
action                       object
passer                       object
receiver                     object
defender                     object
hang_time_secs               object
player_0                     object
player_1                     object
player_2                     object
player_3                     object
player_4                     object
player_5                     object
player_6                     object
player_7                     object
player_8                     object
player_9                     object
player_10                    object
player_11                    object
player_12                    object
player_13                    object
player_14                   

In [73]:
df.line.unique()

array(['D', 'O'], dtype=object)

## Lower case opponent and team columns (make identical)

In [74]:
df['opponent'] = df.opponent.str.lower().str.replace(' ', '', regex=True)

In [75]:
df['opponent']

0         ottawaoutlaws
1         ottawaoutlaws
2         ottawaoutlaws
3         ottawaoutlaws
4         ottawaoutlaws
              ...      
738274      totontorush
738275      totontorush
738276      totontorush
738277      totontorush
738278      totontorush
Name: opponent, Length: 738279, dtype: object

In [76]:
df['team'] = df.team.str.lower().str.replace(' ', '', regex=True).str.replace(',', '', regex=True)

In [77]:
df.team

0                                 NaN
1                                 NaN
2                                 NaN
3                                 NaN
4                                 NaN
                     ...             
738274    newyorkempire2015-stats.csv
738275    newyorkempire2015-stats.csv
738276    newyorkempire2015-stats.csv
738277    newyorkempire2015-stats.csv
738278    newyorkempire2015-stats.csv
Name: team, Length: 738279, dtype: object

#### Regex

In [78]:
df['team'] = df['team'].str.extract('([a-zA-Z ]*)\d*.*')

In [79]:
df.team.unique()

array([nan, 'detroitmechanix', 'sandiegogrowlers', 'dcbreeze',
       'austinsol', 'philadelphiaphoenix', 'tampabaycannons',
       'indianapolisalleycats', 'chicagowildfire', 'dallasroughnecks',
       'pittsburghthunderbirds', 'torontorush', 'newyorkempire',
       'atlantahustle', 'combined', 'seattlecascades',
       'minnesotawindchill', 'raleighflyers', 'madisonradicals',
       'sanjosespiders', 'ottawaoutlaws', 'nashvillenightwatch',
       'sanfranciscoflamethrowers', 'jacksonvillecannons',
       'vancouverriptide', 'cincinnatirevolution', 'losangelesaviators',
       'sfflamethrowers', 'charlotteexpress', 'montrealroyal',
       'rochesterdragons'], dtype=object)

In [80]:
fuzzy_team_list = df.opponent.tolist()

In [81]:
set(fuzzy_team_list)

{'@chicagowildfire',
 '@detroitmechanix',
 '@madisonradicals',
 '@minnesotawindchill',
 '@pittsburghthunderbirds',
 'alleycats',
 'atlanta',
 'atlantahustle',
 'atlantahustle2015',
 'atlantahustle2016',
 'austinsol',
 'austinsol2016',
 'charlotte',
 'charlotteexpress',
 'charlotteexpress2016',
 'chicago',
 'chicagowildfire',
 'chicagowildfre',
 'chicagwildfire',
 'cinci',
 'cincinattirevolution',
 'cincinnati',
 'cincinnatirevolution',
 'cincinnnatirevolution',
 'd.cbreeze',
 'dallas',
 'dallasroughnecks',
 'dallasroughnecks2016',
 'dc',
 'dcbreeze',
 'detroit',
 'detroit.mechanix',
 'detroitmechanics',
 'detroitmechanix',
 'dragons',
 'flame',
 'flamethrowers',
 'indi',
 'indianapolis',
 'indianapolisalleycats',
 'indianapolisalleycays',
 'indianoplisalleycats',
 'indyalleycats',
 'jacksonville',
 'jacksonvillecannons',
 'jacksonvillecannons2015',
 'jacksonvillecannons2016',
 'jacksovillecannons',
 'jun24vssj',
 'la',
 'laaaviators',
 'laaviators',
 'losangelesaviators',
 'losangelesa

In [82]:
longlist= list(df.opponent.unique())

In [83]:
cleanlist = list(df.team.unique())

In [84]:
cleanlist

[nan,
 'detroitmechanix',
 'sandiegogrowlers',
 'dcbreeze',
 'austinsol',
 'philadelphiaphoenix',
 'tampabaycannons',
 'indianapolisalleycats',
 'chicagowildfire',
 'dallasroughnecks',
 'pittsburghthunderbirds',
 'torontorush',
 'newyorkempire',
 'atlantahustle',
 'combined',
 'seattlecascades',
 'minnesotawindchill',
 'raleighflyers',
 'madisonradicals',
 'sanjosespiders',
 'ottawaoutlaws',
 'nashvillenightwatch',
 'sanfranciscoflamethrowers',
 'jacksonvillecannons',
 'vancouverriptide',
 'cincinnatirevolution',
 'losangelesaviators',
 'sfflamethrowers',
 'charlotteexpress',
 'montrealroyal',
 'rochesterdragons']

In [85]:
# Find non-matching elements
def non_match_elements(list_a, list_b):
    non_match = []
    for i in list_a:
        if i not in list_b:
            non_match.append(i)
    return non_match

In [86]:
non_match = non_match_elements(longlist, cleanlist)
print("No match elements: ", non_match)

No match elements:  ['nycempire', 'indianapolisalleycays', 'cincinattirevolution', 'chicagwildfire', 'thunderbirds', 'vsphilly', 'nyempire', 'dc', 'atlanta', 'vspittsburghthunderbirds', '@pittsburghthunderbirds', 'vsatlantahustle', '@chicagowildfire', '@minnesotawindchill', '@madisonradicals', '@detroitmechanix', 'detroitmechanics', 'detroit.mechanix', 'madisonradicals.', 'washingtondcbreeze', 'sjspiders', 'sdgrowlers', 'sanjose', 'sjspiders(5/11)', 'indyalleycats', 'pittsburghthnderbirds', 'mechanix', 'windchill', 'toronto', 'seattlecascades4/6/19', 'laaviators', 'montrealroyals', 'indianapolis', 'tampacannons', 'philadelphiapheonix', 'd.cbreeze', 'indianoplisalleycats', 'pittsburgh', 'ottawaoutlaw', 'montreal', 'washingtondc', 'spiders', 'sandiego', 'flame', 'jacksovillecannons', 'philly', 'vancouver', 'sanfransiscoflamethrowers', 'sjjune10', 'madisonradical', 'jun24vssj', 'la', 'madison', 'madison5.27.17', 'minnesota', 'detroit', 'vs.detroitmechanix', 'vschicagowildfire', 'vsdetroit

In [87]:
clean_teamlist = ['montrealroyal', 'detroitmechanix', 'sandiegogrowlers', 'dcbreeze',
       'austinsol', 'philadelphiaphoenix', 'tampabaycannons',
       'indianapolisalleycats', 'chicagowildfire', 'dallasroughnecks',
       'pittsburghthunderbirds', 'torontorush', 'newyorkempire',
       'atlantahustle', 'seattlecascades', 'minnesotawindchill',
       'raleighflyers', 'madisonradicals', 'sanjosespiders',
       'ottawaoutlaws', 'losangelesaviators']

In [88]:
mat1 = []
for i in fuzzy_team_list:
    mat1.append(process.extract(i, cleanlist, limit=1)[0][0])

df['matches'] = mat1

mat1 = []
for i in fuzzy_team_list:
    mat1.append(process.extract(i, clean_teamlist, limit=1)[0][0])

df['matches'] = mat1  

In [89]:
first_column = df.pop('matches')
# insert column using insert(position,column_name,first_column) function
df.insert(0, 'matches', first_column)

In [90]:
df['opponent_clean'] = df['matches']

In [91]:
df[['opponent_clean', 'opponent', 'team']].loc[df.opponent_clean!=df.opponent]

,opponent_clean,opponent,team
13305,newyorkempire,nycempire,NaN
13306,newyorkempire,nycempire,NaN
13307,newyorkempire,nycempire,NaN
13308,newyorkempire,nycempire,NaN
13309,newyorkempire,nycempire,NaN
...,...,...,...
738274,torontorush,totontorush,newyorkempire
738275,torontorush,totontorush,newyorkempire
738276,torontorush,totontorush,newyorkempire
738277,torontorush,totontorush,newyorkempire


In [92]:
df['check'] = df['opponent_clean'] + '_' + df['opponent']

In [93]:
df['check'].loc[df.opponent_clean!=df.opponent].unique()

array(['newyorkempire_nycempire',
       'indianapolisalleycats_indianapolisalleycays',
       'cincinnatirevolution_cincinattirevolution',
       'chicagowildfire_chicagwildfire',
       'pittsburghthunderbirds_thunderbirds',
       'nashvillenightwatch_vsphilly', 'newyorkempire_nyempire',
       'dcbreeze_dc', 'atlantahustle_atlanta',
       'pittsburghthunderbirds_vspittsburghthunderbirds',
       'pittsburghthunderbirds_@pittsburghthunderbirds',
       'atlantahustle_vsatlantahustle',
       'chicagowildfire_@chicagowildfire',
       'minnesotawindchill_@minnesotawindchill',
       'madisonradicals_@madisonradicals',
       'detroitmechanix_@detroitmechanix',
       'detroitmechanix_detroitmechanics',
       'detroitmechanix_detroit.mechanix',
       'madisonradicals_madisonradicals.', 'dcbreeze_washingtondcbreeze',
       'sanjosespiders_sjspiders', 'sandiegogrowlers_sdgrowlers',
       'sanjosespiders_sanjose', 'sanjosespiders_sjspiders(5/11)',
       'indianapolisalleycats_indya

### Some manual cleaning of wrong matches

In [94]:
df.opponent_clean.value_counts()

madisonradicals              40476
atlantahustle                35377
raleighflyers                34380
minnesotawindchill           34317
pittsburghthunderbirds       34276
newyorkempire                34208
losangelesaviators           34071
dcbreeze                     33157
indianapolisalleycats        33130
torontorush                  32898
montrealroyal                31448
chicagowildfire              31034
seattlecascades              30188
ottawaoutlaws                28896
detroitmechanix              28875
philadelphiaphoenix          28402
dallasroughnecks             28040
sanjosespiders               26690
sandiegogrowlers             25860
austinsol                    25826
nashvillenightwatch          20539
sanfranciscoflamethrowers    19172
tampabaycannons              15382
jacksonvillecannons          13882
vancouverriptide             13320
cincinnatirevolution         10418
charlotteexpress              9129
rochesterdragons              4202
sfflamethrowers     

In [95]:
df['opponent_clean'].loc[df.opponent == 'washingtondc'] = 'dcbreeze'
df['opponent_clean'].loc[df.opponent == 'philly'] = 'philadelphiaphoenix'
df['opponent_clean'].loc[df.opponent == 'sjjune10'] = 'sanjosespiders'
df['opponent_clean'].loc[df.opponent == 'jun24vssj'] = 'sanjosespiders'
df['opponent_clean'].loc[df.opponent == 'la'] = 'losangelesaviators'
df['opponent_clean'].loc[df.opponent == 'vspitt'] = 'pittsburghthunderbirds'
#washingtondc
df['opponent_clean'].loc[df.opponent == 'vsphilly'] = 'philadelphiaphoenix'
df['opponent_clean'].loc[df.opponent == 'vscinci'] = 'cincinnatirevolution'


/Users/ilonanietosvaara/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [96]:
# Drop rows when opponent == test
#df.drop(df[df.opponent== 'test'].index, inplace=True)

In [97]:
df.shape

(738279, 59)

In [98]:
#df['opponent'] = df['matches']

In [99]:
df.drop(['matches', 'check'], axis=1, inplace=True)

In [100]:
df

,datetime,tournamemnt,opponent,point_elapsed_seconds,line,our_score_end_of_point,their_score_end_of_point,event_type,action,passer,...,end_y,distance_unit_of_measure,absolute_distance,lateral_distance,toward_our_goal_distance,team,unnamed:_0,unnamed:_0.1,game_id_str,opponent_clean
0,2019-07-04 19:12,AUDL,ottawaoutlaws,30,D,1,0,Defense,Pull,NaN,...,0.469,yds,94.6,9.1,94.1,NaN,MontrealRoyal2019-stats.csv,NaN,NaN,ottawaoutlaws
1,2019-07-04 19:12,AUDL,ottawaoutlaws,30,D,1,0,Defense,OpponentCatch,NaN,...,0.424,yds,10.7,2.7,-10.3,NaN,MontrealRoyal2019-stats.csv,NaN,NaN,ottawaoutlaws
2,2019-07-04 19:12,AUDL,ottawaoutlaws,30,D,1,0,Defense,OpponentCatch,NaN,...,0.679,yds,18.3,13.6,-12.2,NaN,MontrealRoyal2019-stats.csv,NaN,NaN,ottawaoutlaws
3,2019-07-04 19:12,AUDL,ottawaoutlaws,30,D,1,0,Defense,Throwaway,NaN,...,0.53,yds,18.6,7.9,-16.8,NaN,MontrealRoyal2019-stats.csv,NaN,NaN,ottawaoutlaws
4,2019-07-04 19:12,AUDL,ottawaoutlaws,30,D,1,0,Offense,Catch,Lemieux,...,0.294,yds,13.5,10.7,8.2,NaN,MontrealRoyal2019-stats.csv,NaN,NaN,ottawaoutlaws
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738274,2015-05-09 19:03,AUDL,totontorush,27,O,25,28,Offense,Catch,Skware K,...,NaN,NaN,NaN,NaN,NaN,newyorkempire,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv,torontorush
738275,2015-05-09 19:03,AUDL,totontorush,27,O,25,28,Offense,Catch,Aulett M,...,NaN,NaN,NaN,NaN,NaN,newyorkempire,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv,torontorush
738276,2015-05-09 19:03,AUDL,totontorush,27,O,25,28,Offense,Goal,Marsh J,...,NaN,NaN,NaN,NaN,NaN,newyorkempire,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv,torontorush
738277,2015-05-09 19:03,AUDL,totontorush,19,D,25,28,Defense,Pull,NaN,...,NaN,NaN,NaN,NaN,NaN,newyorkempire,NaN,NaN,2015-05-09 19:03_NewYorkEmpire2015-stats.csv,torontorush


### Create game, team, player, and date IDs

In [101]:
# Drop games agains 'losangelesaviators' (Data corrupt)
#df.drop(df[(df['opponent'] == 'losangelesaviatfors')].index, inplace=True)

In [102]:
df['datetime'] = pd.to_datetime(df['datetime'], format = "%Y/%m/%d")
df['season'] = df['datetime'].dt.year

#Clean season -- some datetime as 2019 
df.loc[df.game_id_str.str.contains('2019'), 'year'] = 2019
df.loc[df.game_id_str.str.contains('2018'), 'year'] = 2018
df.loc[df.game_id_str.str.contains('2017'), 'year'] = 2017
df.loc[df.game_id_str.str.contains('2016'), 'year'] = 2016
df.loc[df.game_id_str.str.contains('2015'), 'year'] = 2015

In [103]:
df.season.unique()

array([2019, 2018, 2017, 2015, 2016])

In [104]:
df.dtypes

datetime                    datetime64[ns]
tournamemnt                         object
opponent                            object
point_elapsed_seconds               object
line                                object
our_score_end_of_point              object
their_score_end_of_point            object
event_type                          object
action                              object
passer                              object
receiver                            object
defender                            object
hang_time_secs                      object
player_0                            object
player_1                            object
player_2                            object
player_3                            object
player_4                            object
player_5                            object
player_6                            object
player_7                            object
player_8                            object
player_9                            object
player_10  

In [105]:
#df['season'] = np.where(df['season'] == 1970, df['year'], df['season'])
#df.drop('year', axis=1, inplace=True)

In [106]:
df.datetime

0        2019-07-04 19:12:00
1        2019-07-04 19:12:00
2        2019-07-04 19:12:00
3        2019-07-04 19:12:00
4        2019-07-04 19:12:00
                 ...        
738274   2015-05-09 19:03:00
738275   2015-05-09 19:03:00
738276   2015-05-09 19:03:00
738277   2015-05-09 19:03:00
738278   2015-05-09 19:03:00
Name: datetime, Length: 738279, dtype: datetime64[ns]

In [107]:
df.season.unique()

array([2019, 2018, 2017, 2015, 2016])

In [108]:
df.opponent_clean.unique()

array(['ottawaoutlaws', 'torontorush', 'dcbreeze', 'philadelphiaphoenix',
       'newyorkempire', 'sanfranciscoflamethrowers', 'madisonradicals',
       'vancouverriptide', 'sandiegogrowlers', 'losangelesaviators',
       'seattlecascades', 'pittsburghthunderbirds',
       'indianapolisalleycats', 'minnesotawindchill', 'detroitmechanix',
       'cincinnatirevolution', 'raleighflyers', 'chicagowildfire',
       'sanjosespiders', 'montrealroyal', 'atlantahustle',
       'tampabaycannons', 'dallasroughnecks', 'austinsol',
       'nashvillenightwatch', 'jacksonvillecannons', 'charlotteexpress',
       'sfflamethrowers', 'rochesterdragons', nan], dtype=object)

In [109]:
df.shape

(738279, 58)

In [110]:
# Drop games from the data for which only one-sided data
df.drop(df[(df['opponent_clean'] == 'losangelesaviators') & (df.season == 2019)].index, inplace=True)
df.drop(df[(df['opponent_clean'] == 'losangelesaviators') & (df.season == 2018)].index, inplace=True)
df.drop(df[(df['opponent_clean'] == 'losangelesaviators') & (df.season == 2017)].index, inplace=True)
df.drop(df[(df['opponent_clean'] == 'madisonradicals') & (df.season == 2017)].index, inplace=True)
df.drop(df[(df['opponent_clean'] == 'montrealroyal') & (df.season == 2016)].index, inplace=True)

In [111]:
df.shape

(701819, 58)

In [112]:
df['game_id_str'].unique()

array([nan, '2019-04-20 18:52_DetroitMechanix2019-stats.csv',
       '2019-04-27 17:48_DetroitMechanix2019-stats.csv', ...,
       '2015-07-03 19:32_NewYorkEmpire2015-stats.csv',
       '2015-05-30 19:18_NewYorkEmpire2015-stats.csv',
       '2015-05-09 19:03_NewYorkEmpire2015-stats.csv'], dtype=object)

In [113]:
#df['game_id_str'] = df['datetime'] + '_' + df['team']
df['game_id'] = df['game_id_str'].astype('category')
df['game_id'] = df['game_id'].cat.codes


In [114]:
df['game_id'].nunique()

1690

In [115]:
df['team_id'] = df['team'].astype('category').cat.codes

In [116]:
df['opponent_id'] = df['opponent_clean'].astype('category').cat.codes

In [117]:
df.team.nunique()

30

In [118]:
df.team_id.nunique()

31

In [119]:
df['datetime'] = pd.to_datetime(df['datetime'], format = "%Y/%m/%d")
df['datetime'] = df['datetime'].dt.date

In [120]:
df['date_id'] = df['datetime'].astype('category').cat.codes

In [121]:
df.date_id.nunique()

238

In [122]:
df[['team', 'team_id', 'opponent','opponent_id', 'datetime', 'date_id']]

,team,team_id,opponent,opponent_id,datetime,date_id
0,NaN,-1,ottawaoutlaws,16,2019-07-04,226
1,NaN,-1,ottawaoutlaws,16,2019-07-04,226
2,NaN,-1,ottawaoutlaws,16,2019-07-04,226
3,NaN,-1,ottawaoutlaws,16,2019-07-04,226
4,NaN,-1,ottawaoutlaws,16,2019-07-04,226
...,...,...,...,...,...,...
738274,newyorkempire,16,totontorush,27,2015-05-09,13
738275,newyorkempire,16,totontorush,27,2015-05-09,13
738276,newyorkempire,16,totontorush,27,2015-05-09,13
738277,newyorkempire,16,totontorush,27,2015-05-09,13


In [123]:
#df['datetime'] = pd.to_datetime(df['datetime'], format = "%Y/%m/%d")

In [124]:
#df['tournamemnt'].loc[df.datetime.dt.year ==1970].unique()

In [125]:
# Number of unique games
df.game_id.nunique()

1690

In [126]:
df.shape

(701819, 62)

## Save semi-clean csv 

In [127]:
df.to_csv(f'{datapath_clean}/semi_clean_cols.csv')

In [128]:
ls ../../clean

EDA_data.csv            grouped_data.csv        semi_clean.csv
all_players.csv         model.csv               semi_clean_cols.csv
cluster_players.csv     reduced_size_clean.csv


## Read in clean data

In [129]:
df = pd.read_csv(f'{datapath_clean}/semi_clean_cols.csv', index_col=None)

/Users/ilonanietosvaara/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (26,27,28,29,30,31,32,33,43,46,49,53,54,56) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [130]:
df.loc[(df.action == 'Callahan') & (~df.receiver.isna())]

,Unnamed: 0,datetime,tournamemnt,opponent,point_elapsed_seconds,line,our_score_end_of_point,their_score_end_of_point,event_type,action,...,team,unnamed:_0,unnamed:_0.1,game_id_str,opponent_clean,season,game_id,team_id,opponent_id,date_id
3722,3722,2019-06-15,AUDL,ottawaoutlaws,70,D,1,1,Offense,Callahan,...,NaN,MontrealRoyal2019-stats.csv,NaN,NaN,ottawaoutlaws,2019,-1,-1,16,219
3949,3949,2019-06-15,AUDL,ottawaoutlaws,36,O,10,11,Offense,Callahan,...,NaN,MontrealRoyal2019-stats.csv,NaN,NaN,ottawaoutlaws,2019,-1,-1,16,219
16052,16052,2017-06-17,AUDL,dcbreeze,164,D,16,14,Offense,Callahan,...,NaN,MontrealRoyal2017-stats.csv,NaN,NaN,dcbreeze,2017,-1,-1,6,130
18793,18793,2017-05-14,AUDL,philadelphiaphoenix,28,O,21,21,Offense,Callahan,...,NaN,MontrealRoyal2017-stats.csv,NaN,NaN,philadelphiaphoenix,2017,-1,-1,17,119
37329,37329,2019-05-18,AUDL,pittsburghthunderbirds,15,O,5,10,Offense,Callahan,...,detroitmechanix,NaN,NaN,2019-05-18 18:03_DetroitMechanix2019-stats.csv,pittsburghthunderbirds,2019,1405,8,18,210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627622,664082,2015-05-09,NaN,jacksonvillecannons2015,7,O,3,10,Offense,Callahan,...,charlotteexpress,NaN,NaN,2015-05-09 10:47_CharlotteExpress2015-stats.csv,jacksonvillecannons,2015,82,2,9,13
645480,681940,2015-05-16,AUDL,charlotteexpress,64,D,0,1,Offense,Callahan,...,nashvillenightwatch,NaN,NaN,2015-05-16 19:01_NashvilleNightwatch2015-stats.csv,charlotteexpress,2015,117,15,2,16
650211,686671,2015-05-03,AUDL,pittsburghthunderbirds,225,O,16,29,Offense,Callahan,...,minnesotawindchill,NaN,NaN,2015-05-03 14:07_MinnesotaWindChill2015-stats.csv,pittsburghthunderbirds,2015,75,13,18,10
654668,691128,2015-05-16,Regular Season Game 1,montrealroyal,32,O,25,24,Offense,Callahan,...,ottawaoutlaws,NaN,NaN,2015-05-16 15:56_OttawaOutlaws2015-stats.csv,montrealroyal,2015,106,17,13,16


In [131]:
df.loc[74060:74071, :]

,Unnamed: 0,datetime,tournamemnt,opponent,point_elapsed_seconds,line,our_score_end_of_point,their_score_end_of_point,event_type,action,...,team,unnamed:_0,unnamed:_0.1,game_id_str,opponent_clean,season,game_id,team_id,opponent_id,date_id
74060,75801,2019-07-27,NaN,indianapolisalleycats,87,O,12,16,Offense,Catch,...,pittsburghthunderbirds,NaN,NaN,2019-07-27 18:58_PittsburghThunderbirds2019-stats.csv,indianapolisalleycats,2019,1684,19,8,236
74061,75802,2019-07-27,NaN,indianapolisalleycats,87,O,12,16,Offense,Catch,...,pittsburghthunderbirds,NaN,NaN,2019-07-27 18:58_PittsburghThunderbirds2019-stats.csv,indianapolisalleycats,2019,1684,19,8,236
74062,75803,2019-07-27,NaN,indianapolisalleycats,87,O,12,16,Offense,Catch,...,pittsburghthunderbirds,NaN,NaN,2019-07-27 18:58_PittsburghThunderbirds2019-stats.csv,indianapolisalleycats,2019,1684,19,8,236
74063,75804,2019-07-27,NaN,indianapolisalleycats,87,O,12,16,Offense,Catch,...,pittsburghthunderbirds,NaN,NaN,2019-07-27 18:58_PittsburghThunderbirds2019-stats.csv,indianapolisalleycats,2019,1684,19,8,236
74064,75805,2019-07-27,NaN,indianapolisalleycats,87,O,12,16,Offense,Catch,...,pittsburghthunderbirds,NaN,NaN,2019-07-27 18:58_PittsburghThunderbirds2019-stats.csv,indianapolisalleycats,2019,1684,19,8,236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74067,75808,2019-07-27,NaN,indianapolisalleycats,87,O,12,16,Offense,Throwaway,...,pittsburghthunderbirds,NaN,NaN,2019-07-27 18:58_PittsburghThunderbirds2019-stats.csv,indianapolisalleycats,2019,1684,19,8,236
74068,75809,2019-07-27,NaN,indianapolisalleycats,87,O,12,16,Defense,Throwaway,...,pittsburghthunderbirds,NaN,NaN,2019-07-27 18:58_PittsburghThunderbirds2019-stats.csv,indianapolisalleycats,2019,1684,19,8,236
74069,75810,2019-07-27,NaN,indianapolisalleycats,87,O,12,16,Offense,Goal,...,pittsburghthunderbirds,NaN,NaN,2019-07-27 18:58_PittsburghThunderbirds2019-stats.csv,indianapolisalleycats,2019,1684,19,8,236
74070,75811,2019-07-27,NaN,indianapolisalleycats,26,D,12,17,Defense,Pull,...,pittsburghthunderbirds,NaN,NaN,2019-07-27 18:58_PittsburghThunderbirds2019-stats.csv,indianapolisalleycats,2019,1684,19,8,236


In [132]:
df.action.unique()

array(['Pull', 'OpponentCatch', 'Throwaway', 'Catch', 'Goal',
       'OpponentPull', 'D', 'OpponentPullOb', 'EndOfFirstQuarter', 'Drop',
       'Halftime', 'EndOfThirdQuarter', 'PullOb', 'GameOver', 'Stall',
       'Callahan', 'EndOfFourthQuarter', 'EndOfOvertime', 'MiscPenalty'],
      dtype=object)

In [133]:
df.columns

Index(['Unnamed: 0', 'datetime', 'tournamemnt', 'opponent',
       'point_elapsed_seconds', 'line', 'our_score_end_of_point',
       'their_score_end_of_point', 'event_type', 'action', 'passer',
       'receiver', 'defender', 'hang_time_secs', 'player_0', 'player_1',
       'player_2', 'player_3', 'player_4', 'player_5', 'player_6', 'player_7',
       'player_8', 'player_9', 'player_10', 'player_11', 'player_12',
       'player_13', 'player_14', 'player_15', 'player_16', 'player_17',
       'player_18', 'player_19', 'player_20', 'player_21', 'player_22',
       'player_23', 'player_24', 'player_25', 'player_26', 'player_27',
       'elapsed_time_secs', 'begin_area', 'begin_x', 'begin_y', 'end_area',
       'end_x', 'end_y', 'distance_unit_of_measure', 'absolute_distance',
       'lateral_distance', 'toward_our_goal_distance', 'team', 'unnamed:_0',
       'unnamed:_0.1', 'game_id_str', 'opponent_clean', 'season', 'game_id',
       'team_id', 'opponent_id', 'date_id'],
      dtype='objec

In [134]:
df.dtypes

Unnamed: 0                int64
datetime                 object
tournamemnt              object
opponent                 object
point_elapsed_seconds     int64
                          ...  
season                    int64
game_id                   int64
team_id                   int64
opponent_id               int64
date_id                   int64
Length: 63, dtype: object

#df.drop(['begin_x', 'begin_area',
       'distance_unit_of_measure','end_x','lateral_distance',
       'end_y', 'absolute_distance', 'toward_our_goal_distance', 'begin_y','end_area'], axis=1, inplace=True)


#Drop player columns
#df.drop(['player_0', 'player_1',
       'player_2', 'player_3', 'player_4', 'player_5', 'player_6', 'player_7',
       'player_8', 'player_9', 'player_10', 'player_11', 'player_12',
       'player_13', 'player_14', 'player_15', 'player_16', 'player_17',
       'player_18', 'player_19', 'player_20', 'player_21', 'player_22',
       'player_23', 'player_24', 'player_25', 'player_26', 'player_27'], axis=1, inplace=True)

In [135]:
df.hang_time_secs.unique()

array([  nan, 7.848, 7.134, ..., 8.502, 9.887, 8.061])

In [136]:
df.shape

(701819, 63)

In [137]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [138]:
df.columns

Index(['datetime', 'tournamemnt', 'opponent', 'point_elapsed_seconds', 'line',
       'our_score_end_of_point', 'their_score_end_of_point', 'event_type',
       'action', 'passer', 'receiver', 'defender', 'hang_time_secs',
       'player_0', 'player_1', 'player_2', 'player_3', 'player_4', 'player_5',
       'player_6', 'player_7', 'player_8', 'player_9', 'player_10',
       'player_11', 'player_12', 'player_13', 'player_14', 'player_15',
       'player_16', 'player_17', 'player_18', 'player_19', 'player_20',
       'player_21', 'player_22', 'player_23', 'player_24', 'player_25',
       'player_26', 'player_27', 'elapsed_time_secs', 'begin_area', 'begin_x',
       'begin_y', 'end_area', 'end_x', 'end_y', 'distance_unit_of_measure',
       'absolute_distance', 'lateral_distance', 'toward_our_goal_distance',
       'team', 'unnamed:_0', 'unnamed:_0.1', 'game_id_str', 'opponent_clean',
       'season', 'game_id', 'team_id', 'opponent_id', 'date_id'],
      dtype='object')

In [142]:

df.drop(['begin_x', 'begin_area',
       'distance_unit_of_measure','end_x','lateral_distance',
       'end_y', 'absolute_distance', 'toward_our_goal_distance', 'begin_y','end_area'], axis=1, inplace=True)

#Drop player columns
df.drop(['player_0', 'player_1',
       'player_2', 'player_3', 'player_4', 'player_5', 'player_6', 'player_7',
       'player_8', 'player_9', 'player_10', 'player_11', 'player_12',
       'player_13', 'player_14', 'player_15', 'player_16', 'player_17',
       'player_18', 'player_19', 'player_20', 'player_21', 'player_22',
       'player_23', 'player_24', 'player_25', 'player_26', 'player_27'], axis=1, inplace=True)

## Save reduced size df to csv

In [143]:
df.to_csv(f'{datapath_clean}/reduced_size_clean2.csv', index=False)

In [146]:
df = pd.read_csv(f'{datapath_clean}/reduced_size_clean2.csv', index_col=None)

In [147]:
df.dtypes

datetime                     object
tournamemnt                  object
opponent                     object
point_elapsed_seconds         int64
line                         object
our_score_end_of_point        int64
their_score_end_of_point      int64
event_type                   object
action                       object
passer                       object
receiver                     object
defender                     object
hang_time_secs              float64
elapsed_time_secs           float64
team                         object
unnamed:_0                   object
unnamed:_0.1                float64
game_id_str                  object
opponent_clean               object
season                        int64
game_id                       int64
team_id                       int64
opponent_id                   int64
date_id                       int64
dtype: object

In [148]:
df.shape

(701819, 24)

In [149]:
df.columns

Index(['datetime', 'tournamemnt', 'opponent', 'point_elapsed_seconds', 'line',
       'our_score_end_of_point', 'their_score_end_of_point', 'event_type',
       'action', 'passer', 'receiver', 'defender', 'hang_time_secs',
       'elapsed_time_secs', 'team', 'unnamed:_0', 'unnamed:_0.1',
       'game_id_str', 'opponent_clean', 'season', 'game_id', 'team_id',
       'opponent_id', 'date_id'],
      dtype='object')

In [150]:
df.line.unique()

array(['D', 'O'], dtype=object)

In [151]:
df.defender.unique()

array(['Joncas', 'Anonymous', nan, ..., 'Wilkes A', 'Awerbu B',
       'Ludwig K'], dtype=object)

In [152]:
df.elapsed_time_secs.unique()

array([0.000000e+00, 4.000000e+00, 8.000000e+00, ..., 1.001400e+04,
       1.021600e+04, 3.300273e+06])

# Feature expansion

## Clean action column

In [153]:
# make dummy column for game over (1 when games over, 0 otherwise)
#df.loc[df['action'] == 'GameOver', 'gameover'] = 1, 0

In [154]:
df['gameover'] = (df.action =='GameOver').astype(int)

In [155]:
df.gameover.unique()

array([0, 1])

In [156]:
df.action.unique()

array(['Pull', 'OpponentCatch', 'Throwaway', 'Catch', 'Goal',
       'OpponentPull', 'D', 'OpponentPullOb', 'EndOfFirstQuarter', 'Drop',
       'Halftime', 'EndOfThirdQuarter', 'PullOb', 'GameOver', 'Stall',
       'Callahan', 'EndOfFourthQuarter', 'EndOfOvertime', 'MiscPenalty'],
      dtype=object)

In [269]:
# Check there are 1505 games
df.gameover.value_counts()

0    552800
1      1505
Name: gameover, dtype: int64

In [270]:
unimp_actions = ['EndOfOvertime', 'EndOfOvertime', 'OpponentCatch', 'Halftime', 
                 'OpponentPull', 'OpponentPullOb', 'EndOfFirstQuarter', 'EndOfThirdQuarter','EndOfFourthQuarter', 'GameOver']

In [271]:
unimp_actions

['EndOfOvertime',
 'EndOfOvertime',
 'OpponentCatch',
 'Halftime',
 'OpponentPull',
 'OpponentPullOb',
 'EndOfFirstQuarter',
 'EndOfThirdQuarter',
 'EndOfFourthQuarter',
 'GameOver']

In [272]:
df.action

0                  Pull
1         OpponentCatch
2         OpponentCatch
3             Throwaway
4                 Catch
              ...      
554300            Catch
554301            Catch
554302             Goal
554303             Pull
554304         GameOver
Name: action, Length: 554305, dtype: object

In [273]:
df.loc[df.action == 'GameOver']

,datetime,tournamemnt,opponent,point_elapsed_seconds,line,our_score_end_of_point,their_score_end_of_point,event_type,action,passer,receiver,defender,hang_time_secs,elapsed_time_secs,team,game_id_str,opponent_clean,season,game_id,team_id,opponent_id,date_id,gameover
882,2019-06-22,AUDL,torontorush,79,O,23,22,Cessation,GameOver,NaN,NaN,NaN,NaN,7448.0,montrealroyal,2019-06-22 17:43_MontrealRoyal2019-stats.csv,torontorush,2019,1473,13,27,221,1
1840,2019-05-26,AUDL,dcbreeze,132,O,17,27,Cessation,GameOver,NaN,NaN,NaN,NaN,7554.0,montrealroyal,2019-05-26 13:00_MontrealRoyal2019-stats.csv,dcbreeze,2019,1429,13,6,213,1
2702,2019-05-27,AUDL,philadelphiaphoenix,35,D,25,15,Cessation,GameOver,NaN,NaN,NaN,NaN,5623.0,montrealroyal,2019-05-27 17:35_MontrealRoyal2019-stats.csv,philadelphiaphoenix,2019,1431,13,17,214,1
4554,2019-07-14,AUDL,philadelphiaphoenix,7,O,20,21,Cessation,GameOver,NaN,NaN,NaN,NaN,8068.0,montrealroyal,2019-07-14 15:10_MontrealRoyal2019-stats.csv,philadelphiaphoenix,2019,1536,13,17,232,1
9476,2018-05-20,AUDL,dcbreeze,7,O,21,20,Cessation,GameOver,NaN,NaN,NaN,NaN,7810.0,montrealroyal,2018-05-20 13:00_MontrealRoyal2018-stats.csv,dcbreeze,2018,1132,13,6,169,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552918,2015-06-21,AUDL,dcbreeze,273,O,21,13,Cessation,GameOver,NaN,NaN,NaN,NaN,7901.0,newyorkempire,2015-06-21 14:09_NewYorkEmpire2015-stats.csv,dcbreeze,2015,254,15,6,35,1
553254,2015-06-28,AUDL,philadelphiaphoenix,38,D,24,13,Cessation,GameOver,NaN,NaN,NaN,NaN,7286.0,newyorkempire,2015-06-28 13:01_NewYorkEmpire2015-stats.csv,philadelphiaphoenix,2015,268,15,17,38,1
553601,2015-07-03,AUDL,rochesterdragons,38,O,25,19,Cessation,GameOver,NaN,NaN,NaN,NaN,8293.0,newyorkempire,2015-07-03 19:32_NewYorkEmpire2015-stats.csv,rochesterdragons,2015,278,15,20,40,1
553999,2015-05-30,AUDL,rochesterdragons,56,D,27,16,Cessation,GameOver,NaN,NaN,NaN,NaN,7569.0,newyorkempire,2015-05-30 19:18_NewYorkEmpire2015-stats.csv,rochesterdragons,2015,176,15,20,25,1


In [274]:
df.loc[1830:1840, :]

,datetime,tournamemnt,opponent,point_elapsed_seconds,line,our_score_end_of_point,their_score_end_of_point,event_type,action,passer,receiver,defender,hang_time_secs,elapsed_time_secs,team,game_id_str,opponent_clean,season,game_id,team_id,opponent_id,date_id,gameover
1830,2019-05-26,AUDL,dcbreeze,132,O,17,27,Offense,Catch,Steve,Saunders,NaN,NaN,7424.0,montrealroyal,2019-05-26 13:00_MontrealRoyal2019-stats.csv,dcbreeze,2019,1429,13,6,213,0
1831,2019-05-26,AUDL,dcbreeze,132,O,17,27,Offense,Catch,Saunders,Steve,NaN,NaN,7433.0,montrealroyal,2019-05-26 13:00_MontrealRoyal2019-stats.csv,dcbreeze,2019,1429,13,6,213,0
1832,2019-05-26,AUDL,dcbreeze,132,O,17,27,Offense,Catch,Steve,Saunders,NaN,NaN,7438.0,montrealroyal,2019-05-26 13:00_MontrealRoyal2019-stats.csv,dcbreeze,2019,1429,13,6,213,0
1833,2019-05-26,AUDL,dcbreeze,132,O,17,27,Offense,Catch,Saunders,Steve,NaN,NaN,7442.0,montrealroyal,2019-05-26 13:00_MontrealRoyal2019-stats.csv,dcbreeze,2019,1429,13,6,213,0
1834,2019-05-26,AUDL,dcbreeze,132,O,17,27,Offense,Throwaway,Steve,Anonymous,NaN,NaN,7449.0,montrealroyal,2019-05-26 13:00_MontrealRoyal2019-stats.csv,dcbreeze,2019,1429,13,6,213,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1836,2019-05-26,AUDL,dcbreeze,132,O,17,27,Offense,Catch,Bonnaud,Quinlan,NaN,NaN,7515.0,montrealroyal,2019-05-26 13:00_MontrealRoyal2019-stats.csv,dcbreeze,2019,1429,13,6,213,0
1837,2019-05-26,AUDL,dcbreeze,132,O,17,27,Offense,Catch,Quinlan,Sokolsky,NaN,NaN,7520.0,montrealroyal,2019-05-26 13:00_MontrealRoyal2019-stats.csv,dcbreeze,2019,1429,13,6,213,0
1838,2019-05-26,AUDL,dcbreeze,132,O,17,27,Offense,Catch,Sokolsky,Charette,NaN,NaN,7524.0,montrealroyal,2019-05-26 13:00_MontrealRoyal2019-stats.csv,dcbreeze,2019,1429,13,6,213,0
1839,2019-05-26,AUDL,dcbreeze,132,O,17,27,Offense,Catch,Charette,Sokolsky,NaN,NaN,7545.0,montrealroyal,2019-05-26 13:00_MontrealRoyal2019-stats.csv,dcbreeze,2019,1429,13,6,213,0


In [275]:
df.shape

(554305, 23)

In [276]:
for act in unimp_actions:
    df.drop(df[df['action'] == f'{act}'].index, inplace=True)

In [277]:
df.shape

(538967, 23)

In [278]:
df.action.unique()

array(['Pull', 'Throwaway', 'Catch', 'Goal', 'D', 'Drop', 'PullOb',
       'Stall', 'Callahan', 'MiscPenalty'], dtype=object)

In [279]:
# Clean out unnecessary actions from action column
#df['action'] = df['action'].str.replace('EndOf.*', '', regex=True)
#df['action'] = df['action'].str.replace('Halftime', '', regex=True)
#df['action'] = df['action'].str.replace('MiscPenalty', '', regex=True)
#df['action'] = df['action'].str.replace('GameOver', '', regex=True)
#df['action'] = df['action'].str.replace('OpponentCatch', '', regex=True)
#df['action'] = df['action'].str.replace('OpponentPullOb', 'Pull', regex=True)
#df['action'] = df['action'].str.replace('OpponentPull', 'Pull', regex=True)
df['action'] = df['action'].str.replace('PullOb', 'Pull', regex=True)


In [280]:
df.action.unique()

array(['Pull', 'Throwaway', 'Catch', 'Goal', 'D', 'Drop', 'Stall',
       'Callahan', 'MiscPenalty'], dtype=object)

In [281]:
#df['action_r'] = df.action.loc[(df.action == 'Pull') | (df.action == 'Throwaway') | (df.action == 'D') | (df.action == 'Goal') | (df.action == 'Drop') | (df.action == 'PullOb') | (df.action == 'Stall') ]

In [282]:
df.head(2)

,datetime,tournamemnt,opponent,point_elapsed_seconds,line,our_score_end_of_point,their_score_end_of_point,event_type,action,passer,receiver,defender,hang_time_secs,elapsed_time_secs,team,game_id_str,opponent_clean,season,game_id,team_id,opponent_id,date_id,gameover
0,2019-07-04,AUDL,ottawaoutlaws,30,D,1,0,Defense,Pull,NaN,NaN,Joncas,NaN,0.0,montrealroyal,2019-07-04 19:12_MontrealRoyal2019-stats.csv,ottawaoutlaws,2019,1512,13,16,226,0
3,2019-07-04,AUDL,ottawaoutlaws,30,D,1,0,Defense,Throwaway,NaN,NaN,Anonymous,NaN,12.0,montrealroyal,2019-07-04 19:12_MontrealRoyal2019-stats.csv,ottawaoutlaws,2019,1512,13,16,226,0


## Show all columns

In [283]:
pd.set_option('display.max_columns', None)

## Create game specific tables with groupby


#### One hot encode

In [284]:
col_transformer_onehot = ColumnTransformer(transformers=[
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'), ['action'])],
                                          remainder='passthrough') 

In [285]:
onehot= pd.DataFrame(col_transformer_onehot.fit_transform(df), columns =col_transformer_onehot.get_feature_names_out())


In [286]:
onehot.columns

Index(['ohe__action_Callahan', 'ohe__action_Catch', 'ohe__action_D',
       'ohe__action_Drop', 'ohe__action_Goal', 'ohe__action_MiscPenalty',
       'ohe__action_Pull', 'ohe__action_Stall', 'ohe__action_Throwaway',
       'remainder__datetime', 'remainder__tournamemnt', 'remainder__opponent',
       'remainder__point_elapsed_seconds', 'remainder__line',
       'remainder__our_score_end_of_point',
       'remainder__their_score_end_of_point', 'remainder__event_type',
       'remainder__passer', 'remainder__receiver', 'remainder__defender',
       'remainder__hang_time_secs', 'remainder__elapsed_time_secs',
       'remainder__team', 'remainder__game_id_str',
       'remainder__opponent_clean', 'remainder__season', 'remainder__game_id',
       'remainder__team_id', 'remainder__opponent_id', 'remainder__date_id',
       'remainder__gameover'],
      dtype='object')

In [287]:
#Clean out columns names - __remainder
#onehot.columns = onehot.columns.str.lstrip(r'^remainder')
#onehot.columns = onehot.columns.str.lstrip(r'^__')

In [288]:
#onehot.drop('ohe__action_', axis=1, inplace=True)

In [289]:
onehot = onehot.rename(columns=lambda x: re.sub('^ohe__action_','',x))
onehot = onehot.rename(columns=lambda x: re.sub('^remainder__','',x))

In [290]:
onehot.columns= onehot.columns.str.lower()

In [291]:
onehot.columns

Index(['callahan', 'catch', 'd', 'drop', 'goal', 'miscpenalty', 'pull',
       'stall', 'throwaway', 'datetime', 'tournamemnt', 'opponent',
       'point_elapsed_seconds', 'line', 'our_score_end_of_point',
       'their_score_end_of_point', 'event_type', 'passer', 'receiver',
       'defender', 'hang_time_secs', 'elapsed_time_secs', 'team',
       'game_id_str', 'opponent_clean', 'season', 'game_id', 'team_id',
       'opponent_id', 'date_id', 'gameover'],
      dtype='object')

### clean datatypes and setup columns for groupby

In [292]:
# Specify event cols that should be summer together
data_cols = onehot.columns[0:9]

In [293]:
data_cols

Index(['callahan', 'catch', 'd', 'drop', 'goal', 'miscpenalty', 'pull',
       'stall', 'throwaway'],
      dtype='object')

In [294]:
onehot.columns[13]

'line'

In [295]:
# Specify columns to take the last 
text_cols = set(onehot.columns) - set(data_cols)

In [296]:
text_cols

{'date_id',
 'datetime',
 'defender',
 'elapsed_time_secs',
 'event_type',
 'game_id',
 'game_id_str',
 'gameover',
 'hang_time_secs',
 'line',
 'opponent',
 'opponent_clean',
 'opponent_id',
 'our_score_end_of_point',
 'passer',
 'point_elapsed_seconds',
 'receiver',
 'season',
 'team',
 'team_id',
 'their_score_end_of_point',
 'tournamemnt'}

In [297]:
onehot.elapsed_time_secs.unique()

array([0.0, 12.0, 20.0, ..., 10014.0, 10216.0, 10662.0], dtype=object)

In [298]:
float_cols = ['point_elapsed_seconds', 'elapsed_time_secs', 'hang_time_secs']

In [299]:
int_cols= list(set(data_cols).union(set(['gameover', 'game_id', 'their_score_end_of_point', 'our_score_end_of_point'])))

In [300]:
onehot[int_cols] = onehot[int_cols].apply(pd.to_numeric, downcast='integer')

In [301]:
onehot[float_cols] = onehot[float_cols].apply(pd.to_numeric)

In [302]:
onehot.dtypes

callahan                       int8
catch                          int8
d                              int8
drop                           int8
goal                           int8
miscpenalty                    int8
pull                           int8
stall                          int8
throwaway                      int8
datetime                     object
tournamemnt                  object
opponent                     object
point_elapsed_seconds         int64
line                         object
our_score_end_of_point         int8
their_score_end_of_point       int8
event_type                   object
passer                       object
receiver                     object
defender                     object
hang_time_secs              float64
elapsed_time_secs           float64
team                         object
game_id_str                  object
opponent_clean               object
season                       object
game_id                       int16
team_id                     

In [303]:
text_cols

{'date_id',
 'datetime',
 'defender',
 'elapsed_time_secs',
 'event_type',
 'game_id',
 'game_id_str',
 'gameover',
 'hang_time_secs',
 'line',
 'opponent',
 'opponent_clean',
 'opponent_id',
 'our_score_end_of_point',
 'passer',
 'point_elapsed_seconds',
 'receiver',
 'season',
 'team',
 'team_id',
 'their_score_end_of_point',
 'tournamemnt'}

In [304]:
# max cols
max_cols = {'their_score_end_of_point','our_score_end_of_point'}

In [305]:
onehot.dtypes

callahan                       int8
catch                          int8
d                              int8
drop                           int8
goal                           int8
miscpenalty                    int8
pull                           int8
stall                          int8
throwaway                      int8
datetime                     object
tournamemnt                  object
opponent                     object
point_elapsed_seconds         int64
line                         object
our_score_end_of_point         int8
their_score_end_of_point       int8
event_type                   object
passer                       object
receiver                     object
defender                     object
hang_time_secs              float64
elapsed_time_secs           float64
team                         object
game_id_str                  object
opponent_clean               object
season                       object
game_id                       int16
team_id                     

In [306]:
d1 = dict.fromkeys(data_cols, 'sum')
d2 = dict.fromkeys(text_cols, 'last')
d_max = dict.fromkeys(max_cols, 'max')
d = {**d1, **d2, **d_max}

In [307]:
#onehot.loc[onehot.game_id == 205]

In [308]:
d

{'callahan': 'sum',
 'catch': 'sum',
 'd': 'sum',
 'drop': 'sum',
 'goal': 'sum',
 'miscpenalty': 'sum',
 'pull': 'sum',
 'stall': 'sum',
 'throwaway': 'sum',
 'event_type': 'last',
 'our_score_end_of_point': 'max',
 'game_id': 'last',
 'team_id': 'last',
 'defender': 'last',
 'datetime': 'last',
 'line': 'last',
 'elapsed_time_secs': 'last',
 'point_elapsed_seconds': 'last',
 'hang_time_secs': 'last',
 'receiver': 'last',
 'team': 'last',
 'opponent_clean': 'last',
 'opponent': 'last',
 'their_score_end_of_point': 'max',
 'opponent_id': 'last',
 'gameover': 'last',
 'passer': 'last',
 'tournamemnt': 'last',
 'game_id_str': 'last',
 'season': 'last',
 'date_id': 'last'}

In [309]:
max(onehot['their_score_end_of_point'])

45

In [310]:
#onehot['win'] = max(onehot['their_score_end_of_point'])

# Groupby

In [311]:
## Attach to passer, defender and receiver team-name (in case players with same names in different teams)
#onehot['passer_team'] = onehot['passer'] + onehot['team']
#onehot['receiver_team']
#onehot['defender_team']

In [312]:
#onehot['passer_team'] = onehot['passer'] + '_' + onehot['team']

In [313]:
#onehot['passer_nan'] = onehot[~np.isnan(onehot['passer'])'passer']

In [314]:
#onehot['passer_id'] = onehot['passer'] + '_' + onehot['game_id'].astype('str')

In [315]:
#onehot['passer'].str.strip().unique()

In [316]:
def clean_string_values(df):
    df['passer'] = df['passer'].str.replace(' ', '_', regex=False).str.lower()
    df['receiver'] = df['receiver'].str.replace(' ', '_', regex=False).str.lower()
    df['defender'] = df['defender'].str.replace(' ', '_', regex=False).str.lower()
    return df[['passer', 'defender', 'receiver']]
    

In [317]:
onehot[['passer', 'defender', 'receiver']] = clean_string_values(onehot) 

In [318]:
onehot['passer_id'] = onehot['passer'] + '_' + onehot['game_id'].astype('str')
onehot['receiver_id'] = onehot['receiver'] + '_' + onehot['game_id'].astype('str')
onehot['defender_id'] = onehot['defender'] + '_' + onehot['game_id'].astype('str')

## Groupby passer

In [319]:
onehot.passer_id.unique()

array([nan, 'lemieux_1512', 'monfette_1512', ..., 'anonymous_100',
       'ivers_b_100', 'awerbu_b_100'], dtype=object)

In [320]:
d

{'callahan': 'sum',
 'catch': 'sum',
 'd': 'sum',
 'drop': 'sum',
 'goal': 'sum',
 'miscpenalty': 'sum',
 'pull': 'sum',
 'stall': 'sum',
 'throwaway': 'sum',
 'event_type': 'last',
 'our_score_end_of_point': 'max',
 'game_id': 'last',
 'team_id': 'last',
 'defender': 'last',
 'datetime': 'last',
 'line': 'last',
 'elapsed_time_secs': 'last',
 'point_elapsed_seconds': 'last',
 'hang_time_secs': 'last',
 'receiver': 'last',
 'team': 'last',
 'opponent_clean': 'last',
 'opponent': 'last',
 'their_score_end_of_point': 'max',
 'opponent_id': 'last',
 'gameover': 'last',
 'passer': 'last',
 'tournamemnt': 'last',
 'game_id_str': 'last',
 'season': 'last',
 'date_id': 'last'}

In [321]:
df_passer = onehot.groupby(['passer_id'], as_index=True).agg(d)

In [322]:
df_passer.shape

(29822, 31)

In [323]:
df_passer.sort_values(by=['game_id', 'passer'], inplace=True)

In [324]:
df_passer.shape

(29822, 31)

In [325]:
df_passer.columns

Index(['callahan', 'catch', 'd', 'drop', 'goal', 'miscpenalty', 'pull',
       'stall', 'throwaway', 'event_type', 'our_score_end_of_point', 'game_id',
       'team_id', 'defender', 'datetime', 'line', 'elapsed_time_secs',
       'point_elapsed_seconds', 'hang_time_secs', 'receiver', 'team',
       'opponent_clean', 'opponent', 'their_score_end_of_point', 'opponent_id',
       'gameover', 'passer', 'tournamemnt', 'game_id_str', 'season',
       'date_id'],
      dtype='object')

In [326]:
# Rename columns
df_passer.rename(columns = {'catch': 'throw', 'drop':'dropped_throw', 'callahan': 'callahan_throw'}, index= {'passer_id': 'act_player_id'}, inplace = True)

In [327]:
#Rename index
df_passer.index.names = ['act_player_id']

In [328]:
#df_passer['act_player'] = df_passer['passer']

In [329]:
# Drop non-passer relevant cols
df_passer.drop(['d', 'pull'], axis = 1, inplace=True)

In [330]:
#Drop rows where passer is anonymous
df_passer = df_passer[df_passer['passer'].str.contains("anonymous") == False]

In [331]:
df_passer.shape

(28935, 29)

In [332]:
df_passer

,callahan_throw,throw,dropped_throw,goal,miscpenalty,stall,throwaway,event_type,our_score_end_of_point,game_id,team_id,defender,datetime,line,elapsed_time_secs,point_elapsed_seconds,hang_time_secs,receiver,team,opponent_clean,opponent,their_score_end_of_point,opponent_id,gameover,passer,tournamemnt,game_id_str,season,date_id
act_player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
chatte_s_0,0,12,0,3,0,0,1,Offense,26,0,22,None,2015-04-11,O,9705.0,271,NaN,anonymous,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25,28,0,chatte_s,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015,0
chialv_f_0,0,45,0,2,0,0,3,Offense,26,0,22,None,2015-04-11,O,9582.0,271,NaN,kerns_e,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25,28,0,chialv_f,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015,0
dallma_l_0,0,12,0,1,0,0,4,Offense,26,0,22,None,2015-04-11,O,9601.0,271,NaN,kerns_e,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25,28,0,dallma_l,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015,0
fein_b_0,0,5,0,0,0,0,0,Offense,26,0,22,None,2015-04-11,O,9357.0,54,NaN,kerns_e,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,24,28,0,fein_b,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015,0
ikeda_c_0,0,11,1,1,0,0,0,Offense,26,0,22,None,2015-04-11,O,9340.0,54,NaN,madden_b,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,24,28,0,ikeda_c,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
matt_b_1551,0,11,0,3,0,0,1,Offense,33,1551,5,None,2019-11-24,D,9109.0,142,NaN,dillon_l,dallasroughnecks,seattlecascades,seattlecascades,26,24,0,matt_b,AUDL,2019-11-24 14:46_DallasRoughnecks2016-stats.csv,2019,237
matty_j_1551,0,3,0,1,0,0,0,Offense,25,1551,5,None,2019-11-24,D,6955.0,258,NaN,jeremy,dallasroughnecks,seattlecascades,seattlecascades,18,24,0,matty_j,AUDL,2019-11-24 14:46_DallasRoughnecks2016-stats.csv,2019,237
muffin_1551,0,46,0,6,0,0,1,Offense,33,1551,5,None,2019-11-24,O,9365.0,269,NaN,cassidy,dallasroughnecks,seattlecascades,seattlecascades,27,24,0,muffin,AUDL,2019-11-24 14:46_DallasRoughnecks2016-stats.csv,2019,237


In [334]:
df_passer.columns

Index(['callahan_throw', 'throw', 'dropped_throw', 'goal', 'miscpenalty',
       'stall', 'throwaway', 'event_type', 'our_score_end_of_point', 'game_id',
       'team_id', 'defender', 'datetime', 'line', 'elapsed_time_secs',
       'point_elapsed_seconds', 'hang_time_secs', 'receiver', 'team',
       'opponent_clean', 'opponent', 'their_score_end_of_point', 'opponent_id',
       'gameover', 'passer', 'tournamemnt', 'game_id_str', 'season',
       'date_id'],
      dtype='object')

## Groupby receiver

In [335]:
rec_cols = {'game_id', 'receiver', 'team_id', 'opponent_id', 'date_id'}
d3 = dict.fromkeys(rec_cols, 'last')
d4 = {**d1, **d3}

In [336]:
df_receiver = onehot.groupby('receiver_id', as_index=True).agg(d4)

In [337]:
df_receiver.sort_values(by=['game_id', 'receiver'], inplace=True)

In [338]:
# Rename columns
df_receiver.rename(columns = {'goal':'goalcatch'}, index= {'receiver_id': 'act_player_id'}, inplace = True)

In [339]:
# Drop non-receiver relevant cols
df_receiver.drop(['throwaway', 'miscpenalty', 'd', 'pull', 'stall', 'callahan'], axis = 1, inplace=True)

In [340]:
df_receiver.index.names = ['act_player_id']

In [341]:
df_receiver.shape

(30673, 8)

In [342]:
df_receiver.loc[(df_receiver['receiver']=='anonymous') & (df_receiver.catch != 0)]

,catch,drop,goalcatch,opponent_id,receiver,game_id,date_id,team_id
act_player_id,,,,,,,,
anonymous_3,4,1,1,27,anonymous,3,0,6
anonymous_5,3,0,0,4,anonymous,5,0,7
anonymous_9,1,0,0,24,anonymous,9,0,10
anonymous_10,1,0,0,7,anonymous,10,0,4
anonymous_11,1,2,1,27,anonymous,11,1,17
...,...,...,...,...,...,...,...,...
anonymous_1516,1,0,1,18,anonymous,1516,228,7
anonymous_1517,1,0,0,6,anonymous,1517,228,27
anonymous_1533,5,0,0,26,anonymous,1533,231,19


In [343]:
#df_receiver['catch'].loc[(df_receiver['receiver'].str.contains('anonymous')) & (df_receiver.line=='O')].sum()

In [344]:
#df_receiver.loc[df_receiver['receiver'].str.contains('anonymous')==False]

In [345]:
#df_receiver['act_player'] = df_receiver['receiver']

In [346]:
# Drop anonymous receiver instances when on defensive line (and the instance is a catch 
df_receiver.drop(df_receiver[(df_receiver['receiver'].str.contains("anonymous") == True)].index, inplace=True)

In [347]:
df_receiver.columns

Index(['catch', 'drop', 'goalcatch', 'opponent_id', 'receiver', 'game_id',
       'date_id', 'team_id'],
      dtype='object')

In [348]:
df_receiver.shape

(29122, 8)

## Groupby defender

In [349]:
def_cols = {'game_id', 'defender','team_id', 'opponent_id', 'date_id'}
d5 = dict.fromkeys(def_cols, 'last')
d6 = {**d1, **d5}

In [350]:
df_defender = onehot.groupby('defender_id', as_index=True).agg(d6)

In [351]:
df_defender.sort_values(by=['game_id', 'defender'], inplace=True)

In [352]:
df_defender.callahan.value_counts()

0    17914
1       75
Name: callahan, dtype: int64

In [353]:
df_defender.columns

Index(['callahan', 'catch', 'd', 'drop', 'goal', 'miscpenalty', 'pull',
       'stall', 'throwaway', 'opponent_id', 'game_id', 'date_id', 'team_id',
       'defender'],
      dtype='object')

In [354]:
df_defender.loc[(df_defender['miscpenalty']!=0)]

,callahan,catch,d,drop,goal,miscpenalty,pull,stall,throwaway,opponent_id,game_id,date_id,team_id,defender
defender_id,,,,,,,,,,,,,,


In [355]:
# Rename columns
#df_defender.rename(columns = {'callahan': 'callahan'}, index= {'receiver_id': 'act_player_id'}, inplace = True)

In [356]:
df_defender.index.names = ['act_player_id']

In [357]:
# Drop non-defender relevant cols
df_defender.drop(['throwaway', 'catch', 'drop', 'miscpenalty', 'goal', 'stall'], axis = 1, inplace=True)

In [358]:
#montreal_full.drop(montreal_full[montreal_full.act_player == 'Anonymous'].index, inplace=True)

In [359]:
df_defender = df_defender[df_defender['defender'].str.contains("anonymous") == False]

In [360]:
#df_defender['act_player'] = df_defender['defender']

In [361]:
df_defender.shape

(16437, 8)

In [362]:
df_defender

,callahan,d,pull,opponent_id,game_id,date_id,team_id,defender
act_player_id,,,,,,,,
chatte_s_0,0,1,0,28,0,0,22,chatte_s
chialv_f_0,0,1,2,28,0,0,22,chialv_f
dallma_l_0,0,1,0,28,0,0,22,dallma_l
fein_b_0,0,1,0,28,0,0,22,fein_b
jeffer_j_0,0,0,7,28,0,0,22,jeffer_j
...,...,...,...,...,...,...,...,...
dylan_1551,0,2,17,24,1551,237,5,dylan
jimmy_1551,0,1,3,24,1551,237,5,jimmy
kurt_1551,0,1,0,24,1551,237,5,kurt


## Merge groupby tables

In [363]:
df = pd.merge(pd.merge(df_receiver, df_defender, on='act_player_id', how='outer'), df_passer, on='act_player_id', how='outer')

In [364]:
df.shape


(29619, 45)

In [365]:
df['act_player'] = df.index

In [366]:
# Create game-independent active player column
df['act_player'].replace(to_replace=r'(_[0-9]+)$', value=r'', regex=True, inplace=True)

In [367]:
df

,catch,drop,goalcatch,opponent_id_x,receiver_x,game_id_x,date_id_x,team_id_x,callahan,d,pull,opponent_id_y,game_id_y,date_id_y,team_id_y,defender_x,callahan_throw,throw,dropped_throw,goal,miscpenalty,stall,throwaway,event_type,our_score_end_of_point,game_id,team_id,defender_y,datetime,line,elapsed_time_secs,point_elapsed_seconds,hang_time_secs,receiver_y,team,opponent_clean,opponent,their_score_end_of_point,opponent_id,gameover,passer,tournamemnt,game_id_str,season,date_id,act_player
act_player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
chatte_s_0,16.0,0.0,2.0,28.0,chatte_s,0.0,0.0,22.0,0.0,1.0,0.0,28.0,0.0,0.0,22.0,chatte_s,0.0,12.0,0.0,3.0,0.0,0.0,1.0,Offense,26.0,0.0,22.0,None,2015-04-11,O,9705.0,271.0,NaN,anonymous,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25.0,28.0,0.0,chatte_s,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,chatte_s
chialv_f_0,27.0,0.0,0.0,28.0,chialv_f,0.0,0.0,22.0,0.0,1.0,2.0,28.0,0.0,0.0,22.0,chialv_f,0.0,45.0,0.0,2.0,0.0,0.0,3.0,Offense,26.0,0.0,22.0,None,2015-04-11,O,9582.0,271.0,NaN,kerns_e,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25.0,28.0,0.0,chialv_f,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,chialv_f
dallma_l_0,18.0,0.0,4.0,28.0,dallma_l,0.0,0.0,22.0,0.0,1.0,0.0,28.0,0.0,0.0,22.0,dallma_l,0.0,12.0,0.0,1.0,0.0,0.0,4.0,Offense,26.0,0.0,22.0,None,2015-04-11,O,9601.0,271.0,NaN,kerns_e,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25.0,28.0,0.0,dallma_l,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,dallma_l
fein_b_0,5.0,0.0,0.0,28.0,fein_b,0.0,0.0,22.0,0.0,1.0,0.0,28.0,0.0,0.0,22.0,fein_b,0.0,5.0,0.0,0.0,0.0,0.0,0.0,Offense,26.0,0.0,22.0,None,2015-04-11,O,9357.0,54.0,NaN,kerns_e,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,24.0,28.0,0.0,fein_b,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,fein_b
ikeda_c_0,10.0,0.0,0.0,28.0,ikeda_c,0.0,0.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.0,1.0,1.0,0.0,0.0,0.0,Offense,26.0,0.0,22.0,None,2015-04-11,O,9340.0,54.0,NaN,madden_b,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,24.0,28.0,0.0,ikeda_c,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,ikeda_c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
perivier_1503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Offense,8.0,1503.0,0.0,None,2019-06-29,D,2208.0,124.0,NaN,trenton,atlantahustle,dallasroughnecks,dallasroughnecks,7.0,5.0,0.0,perivier,None,2019-06-29 21:29_AtlantaHustle2019-stats.csv,2019.0,224.0,perivier
brodbeck_1507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Offense,12.0,1507.0,1.0,None,2019-06-30,O,2604.0,271.0,NaN,fan_d,austinsol,atlantahustle,atlantahustle,10.0,0.0,0.0,brodbeck,None,2019-06-30 13:09_AustinSol2019-stats.csv,2019.0,225.0,brodbeck
mika_c_1507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Offense,2.0,1507.0,1.0,None,2019-06-30,D,144.0,27.0,NaN,purcell,austinsol,atlantahustle,atlantahustle,0.0,0.0,0.0,mika_c,None,2019-06-30 13:09_AustinSol2019-stats.csv,2019.0,225.0,mika_c


In [368]:
df.columns

Index(['catch', 'drop', 'goalcatch', 'opponent_id_x', 'receiver_x',
       'game_id_x', 'date_id_x', 'team_id_x', 'callahan', 'd', 'pull',
       'opponent_id_y', 'game_id_y', 'date_id_y', 'team_id_y', 'defender_x',
       'callahan_throw', 'throw', 'dropped_throw', 'goal', 'miscpenalty',
       'stall', 'throwaway', 'event_type', 'our_score_end_of_point', 'game_id',
       'team_id', 'defender_y', 'datetime', 'line', 'elapsed_time_secs',
       'point_elapsed_seconds', 'hang_time_secs', 'receiver_y', 'team',
       'opponent_clean', 'opponent', 'their_score_end_of_point', 'opponent_id',
       'gameover', 'passer', 'tournamemnt', 'game_id_str', 'season', 'date_id',
       'act_player'],
      dtype='object')

In [369]:
df[['date_id_x', 'date_id_y', 'date_id' 
]]

,date_id_x,date_id_y,date_id
act_player_id,,,
chatte_s_0,0.0,0.0,0.0
chialv_f_0,0.0,0.0,0.0
dallma_l_0,0.0,0.0,0.0
fein_b_0,0.0,0.0,0.0
ikeda_c_0,0.0,NaN,0.0
...,...,...,...
perivier_1503,NaN,NaN,224.0
brodbeck_1507,NaN,NaN,225.0
mika_c_1507,NaN,NaN,225.0


In [370]:
df[['date_id_y', 'date_id_x']].loc[(df.date_id_x != df.date_id_y) & (~df.date_id_x.isna()) & (~df.date_id_y.isna())]

,date_id_y,date_id_x
act_player_id,,


In [371]:
df[['date_id', 'date_id_y']].loc[(df.date_id_y != df.date_id) & (~df.date_id_y.isna()) & (~df.date_id.isna())]

,date_id,date_id_y
act_player_id,,


### Re-merge id columns

In [372]:
df.columns

Index(['catch', 'drop', 'goalcatch', 'opponent_id_x', 'receiver_x',
       'game_id_x', 'date_id_x', 'team_id_x', 'callahan', 'd', 'pull',
       'opponent_id_y', 'game_id_y', 'date_id_y', 'team_id_y', 'defender_x',
       'callahan_throw', 'throw', 'dropped_throw', 'goal', 'miscpenalty',
       'stall', 'throwaway', 'event_type', 'our_score_end_of_point', 'game_id',
       'team_id', 'defender_y', 'datetime', 'line', 'elapsed_time_secs',
       'point_elapsed_seconds', 'hang_time_secs', 'receiver_y', 'team',
       'opponent_clean', 'opponent', 'their_score_end_of_point', 'opponent_id',
       'gameover', 'passer', 'tournamemnt', 'game_id_str', 'season', 'date_id',
       'act_player'],
      dtype='object')

In [373]:
df.date_id.isna().sum()

684

In [374]:
df.team_id.isna().sum()

684

In [375]:
df.game_id.isna().sum()

684

In [376]:
df.opponent_id.isna().sum()

684

In [377]:
# Date id
df['date_id'] = df['date_id'].replace('', pd.NA).fillna(df['date_id_y'])
df['date_id'] = df['date_id'].replace('', pd.NA).fillna(df['date_id_x'])

In [378]:
# Team id
df['team_id'] = df['team_id'].replace('', pd.NA).fillna(df['team_id_y'])
df['team_id'] = df['team_id'].replace('', pd.NA).fillna(df['team_id_x'])

In [379]:
# Team id
df['game_id'] = df['game_id'].replace('', pd.NA).fillna(df['game_id_y'])
df['game_id'] = df['game_id'].replace('', pd.NA).fillna(df['game_id_x'])

In [380]:
# Team id
df['opponent_id'] = df['opponent_id'].replace('', pd.NA).fillna(df['opponent_id_y'])
df['opponent_id'] = df['opponent_id'].replace('', pd.NA).fillna(df['opponent_id_x'])

In [381]:
# Drop superfluos columns

df.drop(['date_id_y', 'date_id_x', 'team_id_x', 'team_id_y', 'game_id_y', 'game_id_x', 'opponent_id_x', 'opponent_id_y', 'receiver_y', 'receiver_x', 'defender_x', 'defender_y','passer'], axis = 1, inplace =True)

In [382]:
df.columns

Index(['catch', 'drop', 'goalcatch', 'callahan', 'd', 'pull', 'callahan_throw',
       'throw', 'dropped_throw', 'goal', 'miscpenalty', 'stall', 'throwaway',
       'event_type', 'our_score_end_of_point', 'game_id', 'team_id',
       'datetime', 'line', 'elapsed_time_secs', 'point_elapsed_seconds',
       'hang_time_secs', 'team', 'opponent_clean', 'opponent',
       'their_score_end_of_point', 'opponent_id', 'gameover', 'tournamemnt',
       'game_id_str', 'season', 'date_id', 'act_player'],
      dtype='object')

### Save csv - groupedby

In [383]:
df.to_csv(f'{datapath_clean}/grouped_data.csv')

In [384]:
# reas in csv
df = pd.read_csv(f'{datapath_clean}/grouped_data.csv')

In [385]:
df

,act_player_id,catch,drop,goalcatch,callahan,d,pull,callahan_throw,throw,dropped_throw,goal,miscpenalty,stall,throwaway,event_type,our_score_end_of_point,game_id,team_id,datetime,line,elapsed_time_secs,point_elapsed_seconds,hang_time_secs,team,opponent_clean,opponent,their_score_end_of_point,opponent_id,gameover,tournamemnt,game_id_str,season,date_id,act_player
0,chatte_s_0,16.0,0.0,2.0,0.0,1.0,0.0,0.0,12.0,0.0,3.0,0.0,0.0,1.0,Offense,26.0,0.0,22.0,2015-04-11,O,9705.0,271.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,chatte_s
1,chialv_f_0,27.0,0.0,0.0,0.0,1.0,2.0,0.0,45.0,0.0,2.0,0.0,0.0,3.0,Offense,26.0,0.0,22.0,2015-04-11,O,9582.0,271.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,chialv_f
2,dallma_l_0,18.0,0.0,4.0,0.0,1.0,0.0,0.0,12.0,0.0,1.0,0.0,0.0,4.0,Offense,26.0,0.0,22.0,2015-04-11,O,9601.0,271.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,dallma_l
3,fein_b_0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,Offense,26.0,0.0,22.0,2015-04-11,O,9357.0,54.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,24.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,fein_b
4,ikeda_c_0,10.0,0.0,0.0,NaN,NaN,NaN,0.0,11.0,1.0,1.0,0.0,0.0,0.0,Offense,26.0,0.0,22.0,2015-04-11,O,9340.0,54.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,24.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,ikeda_c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29614,perivier_1503,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Offense,8.0,1503.0,0.0,2019-06-29,D,2208.0,124.0,NaN,atlantahustle,dallasroughnecks,dallasroughnecks,7.0,5.0,0.0,NaN,2019-06-29 21:29_AtlantaHustle2019-stats.csv,2019.0,224.0,perivier
29615,brodbeck_1507,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Offense,12.0,1507.0,1.0,2019-06-30,O,2604.0,271.0,NaN,austinsol,atlantahustle,atlantahustle,10.0,0.0,0.0,NaN,2019-06-30 13:09_AustinSol2019-stats.csv,2019.0,225.0,brodbeck
29616,mika_c_1507,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Offense,2.0,1507.0,1.0,2019-06-30,D,144.0,27.0,NaN,austinsol,atlantahustle,atlantahustle,0.0,0.0,0.0,NaN,2019-06-30 13:09_AustinSol2019-stats.csv,2019.0,225.0,mika_c
29617,wolfe_j_1507,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Offense,21.0,1507.0,1.0,2019-06-30,O,6829.0,43.0,NaN,austinsol,atlantahustle,atlantahustle,22.0,0.0,0.0,NaN,2019-06-30 13:09_AustinSol2019-stats.csv,2019.0,225.0,wolfe_j


### Create additional variables

In [386]:
df.columns

Index(['act_player_id', 'catch', 'drop', 'goalcatch', 'callahan', 'd', 'pull',
       'callahan_throw', 'throw', 'dropped_throw', 'goal', 'miscpenalty',
       'stall', 'throwaway', 'event_type', 'our_score_end_of_point', 'game_id',
       'team_id', 'datetime', 'line', 'elapsed_time_secs',
       'point_elapsed_seconds', 'hang_time_secs', 'team', 'opponent_clean',
       'opponent', 'their_score_end_of_point', 'opponent_id', 'gameover',
       'tournamemnt', 'game_id_str', 'season', 'date_id', 'act_player'],
      dtype='object')

In [387]:
df['throws_all'] = df[['throw','throwaway', 'goal', 'dropped_throw', 'callahan_throw']].sum(axis=1)

In [388]:
df[['act_player_id', 'throws_all', 'throw','throwaway', 'goal', 'dropped_throw', 'callahan_throw']]

,act_player_id,throws_all,throw,throwaway,goal,dropped_throw,callahan_throw
0,chatte_s_0,16.0,12.0,1.0,3.0,0.0,0.0
1,chialv_f_0,50.0,45.0,3.0,2.0,0.0,0.0
2,dallma_l_0,17.0,12.0,4.0,1.0,0.0,0.0
3,fein_b_0,5.0,5.0,0.0,0.0,0.0,0.0
4,ikeda_c_0,13.0,11.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...
29614,perivier_1503,1.0,1.0,0.0,0.0,0.0,0.0
29615,brodbeck_1507,1.0,1.0,0.0,0.0,0.0,0.0
29616,mika_c_1507,1.0,1.0,0.0,0.0,0.0,0.0
29617,wolfe_j_1507,1.0,0.0,1.0,0.0,0.0,0.0


In [389]:
df['turnover'] = df[['callahan_throw', 'miscpenalty', 'stall', 'throwaway']].sum(axis=1)

In [390]:
df[['turnover', 'act_player_id', 'miscpenalty', 'stall', 'throwaway', 'throws_all', 'throw','throwaway', 'goal', 'dropped_throw', 'callahan_throw']]

,turnover,act_player_id,miscpenalty,stall,throwaway,throws_all,throw,throwaway,goal,dropped_throw,callahan_throw
0,1.0,chatte_s_0,0.0,0.0,1.0,16.0,12.0,1.0,3.0,0.0,0.0
1,3.0,chialv_f_0,0.0,0.0,3.0,50.0,45.0,3.0,2.0,0.0,0.0
2,4.0,dallma_l_0,0.0,0.0,4.0,17.0,12.0,4.0,1.0,0.0,0.0
3,0.0,fein_b_0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0
4,0.0,ikeda_c_0,0.0,0.0,0.0,13.0,11.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
29614,0.0,perivier_1503,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
29615,0.0,brodbeck_1507,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
29616,0.0,mika_c_1507,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
29617,1.0,wolfe_j_1507,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


In [391]:
#df[['act_player_id', 'drop', 'catch', 'catch_perc', 'pass_perc']]

In [392]:
value_true = (df['throws_all'] - df['turnover']) / df['throws_all']

In [393]:
value_no_true = np.nan

In [394]:
try:
    df['pass_perc'] = np.where(df['throws_all'].notnull(), value_true, value_no_true)
except:
    print('didnt work')

In [395]:
# Replace infinite values with nan
df.loc[(~np.isfinite(df['pass_perc'])) & (df.pass_perc.notnull()), 'pass_perc'] = np.nan

In [396]:
df.pass_perc.describe()

count    28929.000000
mean         0.908042
std          0.144454
min         -1.000000
25%          0.875000
50%          0.952381
75%          1.000000
max          1.000000
Name: pass_perc, dtype: float64

In [397]:
# Succesful catches: catches ÷ (catches + drops)
value_true2 = df['catch'] / (df['catch'] + df['drop'])
value_no_true2 = np.nan

In [398]:
try:
    df['catch_perc'] = np.where((df['catch'].notnull()) | (df['drop'].notnull()), value_true2, value_no_true2)
except:
    print('didnt work')

In [399]:
df['catch_perc'].describe()

count    28655.000000
mean         0.969488
std          0.105049
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: catch_perc, dtype: float64

In [400]:
df.loc[df.catch_perc ==1]

,act_player_id,catch,drop,goalcatch,callahan,d,pull,callahan_throw,throw,dropped_throw,goal,miscpenalty,stall,throwaway,event_type,our_score_end_of_point,game_id,team_id,datetime,line,elapsed_time_secs,point_elapsed_seconds,hang_time_secs,team,opponent_clean,opponent,their_score_end_of_point,opponent_id,gameover,tournamemnt,game_id_str,season,date_id,act_player,throws_all,turnover,pass_perc,catch_perc
0,chatte_s_0,16.0,0.0,2.0,0.0,1.0,0.0,0.0,12.0,0.0,3.0,0.0,0.0,1.0,Offense,26.0,0.0,22.0,2015-04-11,O,9705.0,271.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,chatte_s,16.0,1.0,0.937500,1.0
1,chialv_f_0,27.0,0.0,0.0,0.0,1.0,2.0,0.0,45.0,0.0,2.0,0.0,0.0,3.0,Offense,26.0,0.0,22.0,2015-04-11,O,9582.0,271.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,chialv_f,50.0,3.0,0.940000,1.0
2,dallma_l_0,18.0,0.0,4.0,0.0,1.0,0.0,0.0,12.0,0.0,1.0,0.0,0.0,4.0,Offense,26.0,0.0,22.0,2015-04-11,O,9601.0,271.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,dallma_l,17.0,4.0,0.764706,1.0
3,fein_b_0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,Offense,26.0,0.0,22.0,2015-04-11,O,9357.0,54.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,24.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,fein_b,5.0,0.0,1.000000,1.0
4,ikeda_c_0,10.0,0.0,0.0,NaN,NaN,NaN,0.0,11.0,1.0,1.0,0.0,0.0,0.0,Offense,26.0,0.0,22.0,2015-04-11,O,9340.0,54.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,24.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,ikeda_c,13.0,0.0,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29113,jimmy_1551,27.0,0.0,4.0,0.0,1.0,3.0,0.0,20.0,1.0,4.0,0.0,0.0,2.0,Offense,33.0,1551.0,5.0,2019-11-24,O,9393.0,269.0,NaN,dallasroughnecks,seattlecascades,seattlecascades,27.0,24.0,0.0,AUDL,2019-11-24 14:46_DallasRoughnecks2016-stats.csv,2019.0,237.0,jimmy,27.0,2.0,0.925926,1.0
29116,matt_b_1551,12.0,0.0,0.0,0.0,0.0,14.0,0.0,11.0,0.0,3.0,0.0,0.0,1.0,Offense,33.0,1551.0,5.0,2019-11-24,D,9109.0,142.0,NaN,dallasroughnecks,seattlecascades,seattlecascades,26.0,24.0,0.0,AUDL,2019-11-24 14:46_DallasRoughnecks2016-stats.csv,2019.0,237.0,matt_b,15.0,1.0,0.933333,1.0
29117,matty_j_1551,4.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,Offense,25.0,1551.0,5.0,2019-11-24,D,6955.0,258.0,NaN,dallasroughnecks,seattlecascades,seattlecascades,18.0,24.0,0.0,AUDL,2019-11-24 14:46_DallasRoughnecks2016-stats.csv,2019.0,237.0,matty_j,4.0,0.0,1.000000,1.0
29118,muffin_1551,38.0,0.0,0.0,NaN,NaN,NaN,0.0,46.0,0.0,6.0,0.0,0.0,1.0,Offense,33.0,1551.0,5.0,2019-11-24,O,9365.0,269.0,NaN,dallasroughnecks,seattlecascades,seattlecascades,27.0,24.0,0.0,AUDL,2019-11-24 14:46_DallasRoughnecks2016-stats.csv,2019.0,237.0,muffin,53.0,1.0,0.981132,1.0


### Groupby game_id 




### Add ranking (manually based on: https://theaudl.com/league/power-rankings/2019-year-review

In [401]:
df['ranking2019'] = 0

In [402]:
df.team.unique()

array(['sanfranciscoflamethrowers', 'vancouverriptide', 'torontorush',
       'dcbreeze', 'raleighflyers', 'detroitmechanix',
       'indianapolisalleycats', 'pittsburghthunderbirds',
       'seattlecascades', 'losangelesaviators', 'cincinnatirevolution',
       'philadelphiaphoenix', 'montrealroyal', 'newyorkempire',
       'charlotteexpress', 'sanjosespiders', 'sandiegogrowlers',
       'jacksonvillecannons', nan, 'nashvillenightwatch',
       'madisonradicals', 'ottawaoutlaws', 'minnesotawindchill',
       'chicagowildfire', 'rochesterdragons', 'sfflamethrowers',
       'dallasroughnecks', 'atlantahustle', 'austinsol',
       'tampabaycannons'], dtype=object)

In [403]:
df['ranking2019'].loc[df.team == 'detroitmechanix'] = 21
df['ranking2019'].loc[df.team == 'seattlecascades'] = 20
df['ranking2019'].loc[df.team == 'sanjosespiders'] = 19
df['ranking2019'].loc[df.team == 'austinsol'] = 18
df['ranking2019'].loc[df.team == 'ottawaoutlaws'] = 17
df['ranking2019'].loc[df.team == 'tampabaycannons'] = 16
df['ranking2019'].loc[df.team == 'montrealroyal'] = 15
df['ranking2019'].loc[df.team == 'philadelphiaphoenix'] = 14
df['ranking2019'].loc[df.team == 'minnesotawindchill'] = 13
df['ranking2019'].loc[df.team == 'madisonradicals'] = 12
df['ranking2019'].loc[df.team == 'atlantahustle'] = 11
df['ranking2019'].loc[df.team == 'chicagowildfire'] = 10
df['ranking2019'].loc[df.team == 'pittsburghthunderbirds'] = 8
df['ranking2019'].loc[df.team == 'dcbreeze'] = 7
df['ranking2019'].loc[df.team == 'torontorush'] = 6
df['ranking2019'].loc[df.team == 'indianapolisalleycats'] = 5
df['ranking2019'].loc[df.team == 'raleighflyers'] = 4
df['ranking2019'].loc[df.team == 'sandiegogrowlers'] = 3
df['ranking2019'].loc[df.team == 'dallasroughnecks'] = 2
df['ranking2019'].loc[df.team == 'newyorkempire'] = 1

/Users/ilonanietosvaara/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [404]:
df.ranking2019.unique()

array([ 0,  6,  7,  4, 21,  5,  8, 20, 14, 15,  1, 19,  3, 12, 17, 13, 10,
        2, 11, 18, 16])

In [405]:
df

,act_player_id,catch,drop,goalcatch,callahan,d,pull,callahan_throw,throw,dropped_throw,goal,miscpenalty,stall,throwaway,event_type,our_score_end_of_point,game_id,team_id,datetime,line,elapsed_time_secs,point_elapsed_seconds,hang_time_secs,team,opponent_clean,opponent,their_score_end_of_point,opponent_id,gameover,tournamemnt,game_id_str,season,date_id,act_player,throws_all,turnover,pass_perc,catch_perc,ranking2019
0,chatte_s_0,16.0,0.0,2.0,0.0,1.0,0.0,0.0,12.0,0.0,3.0,0.0,0.0,1.0,Offense,26.0,0.0,22.0,2015-04-11,O,9705.0,271.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,chatte_s,16.0,1.0,0.937500,1.0,0
1,chialv_f_0,27.0,0.0,0.0,0.0,1.0,2.0,0.0,45.0,0.0,2.0,0.0,0.0,3.0,Offense,26.0,0.0,22.0,2015-04-11,O,9582.0,271.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,chialv_f,50.0,3.0,0.940000,1.0,0
2,dallma_l_0,18.0,0.0,4.0,0.0,1.0,0.0,0.0,12.0,0.0,1.0,0.0,0.0,4.0,Offense,26.0,0.0,22.0,2015-04-11,O,9601.0,271.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,25.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,dallma_l,17.0,4.0,0.764706,1.0,0
3,fein_b_0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,Offense,26.0,0.0,22.0,2015-04-11,O,9357.0,54.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,24.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,fein_b,5.0,0.0,1.000000,1.0,0
4,ikeda_c_0,10.0,0.0,0.0,NaN,NaN,NaN,0.0,11.0,1.0,1.0,0.0,0.0,0.0,Offense,26.0,0.0,22.0,2015-04-11,O,9340.0,54.0,NaN,sanfranciscoflamethrowers,vancouverriptide,vancouverriptide,24.0,28.0,0.0,AUDL West 2015,2015-04-11 18:21_SanFranciscoFlameThrowers2015-stats.csv,2015.0,0.0,ikeda_c,13.0,0.0,1.000000,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29614,perivier_1503,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Offense,8.0,1503.0,0.0,2019-06-29,D,2208.0,124.0,NaN,atlantahustle,dallasroughnecks,dallasroughnecks,7.0,5.0,0.0,NaN,2019-06-29 21:29_AtlantaHustle2019-stats.csv,2019.0,224.0,perivier,1.0,0.0,1.000000,NaN,11
29615,brodbeck_1507,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Offense,12.0,1507.0,1.0,2019-06-30,O,2604.0,271.0,NaN,austinsol,atlantahustle,atlantahustle,10.0,0.0,0.0,NaN,2019-06-30 13:09_AustinSol2019-stats.csv,2019.0,225.0,brodbeck,1.0,0.0,1.000000,NaN,18
29616,mika_c_1507,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Offense,2.0,1507.0,1.0,2019-06-30,D,144.0,27.0,NaN,austinsol,atlantahustle,atlantahustle,0.0,0.0,0.0,NaN,2019-06-30 13:09_AustinSol2019-stats.csv,2019.0,225.0,mika_c,1.0,0.0,1.000000,NaN,18
29617,wolfe_j_1507,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Offense,21.0,1507.0,1.0,2019-06-30,O,6829.0,43.0,NaN,austinsol,atlantahustle,atlantahustle,22.0,0.0,0.0,NaN,2019-06-30 13:09_AustinSol2019-stats.csv,2019.0,225.0,wolfe_j,1.0,1.0,0.000000,NaN,18


### Save to clean

In [406]:
df.to_csv(f'{datapath_clean}/EDA_data.csv', index= False)

#### Read EDA csv

In [ ]:
df = pd.read_csv(f'{datapath_clean}/EDA_data.csv', index_col=None)

### game column

In [ ]:
agg_cols = {'callahan',
 'callahan_throw',
 'catch',
 'catch_perc',
 'd',
 'drop',
 'dropped_throw',
 'goal',
 'goalcatch',
 'hang_time_secs',
 'miscpenalty',
 'pass_perc',
 'pull',
 'stall',
 'throw',
 'throwaway',
 'throws_all',
 'turnover'}

In [ ]:
d1_team = dict.fromkeys(agg_cols, 'sum')
d2_team = dict.fromkeys(last_cols, 'last')
d_team = {**d1_team, **d2_team}

In [ ]:
last_cols = {'act_player_id','game_id_x','game_id_y', 'gameover', 'point_elapsed_seconds',
       'game_id', 'line', 'team', 'their_score_end_of_point', 'opponent', 'game_id_str',
       'our_score_end_of_point', 'datetime', 'elapsed_time_secs',
       'tournamemnt', 'ranking2019'}

In [ ]:
last_cols

In [ ]:
df_team = df.groupby('team').agg(d_team)
#as_index=False

In [ ]:
df_team

### Just one dataframe of games - groupby game

In [ ]:
df_last['win'] = np.where(df_last['our_score_end_of_point'] > df_last['their_score_end_of_point'], 0,1)

### Create win / loss column

In [ ]:
df_last = df.groupby('game_id').tail(1)

In [ ]:
df_last.reset_index(drop=True, inplace=True)